In [ ]:
import pandas as pd
from openprompt.data_utils import InputExample
import os
import re
# ==============================
# Load paraphrase train CSV
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\PARA\train_pairs.csv")

# ==============================
# Label mapping
# ==============================
label_map = {"P": 1, "NP": 0}
classes = list(label_map.keys())

guid = 0
examples = []
class_stats = {}

# ==============================
# Helper: Extract inner text from XML
# ==============================


def read_xml_text(path):
    """
    Reads an XML file and returns only the cleaned text inside <UPPC_document>...</UPPC_document>.
    Removes all tags and converts inner double quotes to single quotes.
    """
    with open(path, "r", encoding="utf-8") as f:
        xml = f.read()
    
    # Extract text between opening and closing <UPPC_document>
    start = xml.find(">") + 1
    end = xml.rfind("</UPPC_document>")
    text = xml[start:end].strip()
    
    # Remove any remaining tags inside the text
    text = re.sub(r"<.*?>", "", text)
    
    # Replace multiple spaces/newlines with single space
    text = re.sub(r"\s+", " ", text).strip()
    
    # Replace double quotes inside text with single quotes
    text = text.replace('"', "'")
    
    return text

# ==============================
# Take first 16 examples per class
# ==============================
for label_name in classes:
    class_df = df[df["label"] == label_name].head(16)
    class_stats[label_name] = len(class_df)

    for _, row in class_df.iterrows():
        source_file = os.path.join(r"C:\Users\stdFurqan\Desktop\paft\PARA\data", row['source_file'])
        target_file = os.path.join(r"C:\Users\stdFurqan\Desktop\paft\PARA\data", row['target_file'])

        text_a = read_xml_text(source_file)
        text_b = read_xml_text(target_file)

        examples.append(
            InputExample(
                guid=guid,
                text_a=text_a,
                text_b=text_b,
                label=label_map[label_name]
            )
        )
        guid += 1

# ==============================
# Print InputExamples in full
# ==============================
for ex in examples:
    print("InputExample(")
    print(f"    guid={ex.guid},")
    print(f"    text_a=\"{ex.text_a}\",")
    print(f"    text_b=\"{ex.text_b}\",")
    print(f"    label={ex.label}")
    print("),")

# ==============================
# Metadata
# ==============================
print("\n### Classes ###")
print(f"classes = {classes}")

print("\n### Label Map ###")
print(f"label_map = {label_map}")

print("\n### Samples per Class ###")
for cls, count in class_stats.items():
    print(f"{cls}: {count}")

print(f"\nTotal InputExamples generated: {guid}")


In [ ]:
templates = [
    ("T1", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کیا ایک جیسے ہیں؟ {"mask"}',
        tokenizer=tokenizer,
    )),

    ("T2", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} میں معنی کی مطابقت {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("T3", ManualTemplate(
        text='کیا یہ دونوں جملے {"placeholder":"text_a"} اور {"placeholder":"text_b"} ایک دوسرے کی تکرار ہیں؟ {"mask"}',
        tokenizer=tokenizer,
    )),

    ("T4", ManualTemplate(
        text='{"placeholder":"text_a"} کے مطابق {"placeholder":"text_b"} {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("T5", ManualTemplate(
        text='دونوں جملے {"placeholder":"text_a"} اور {"placeholder":"text_b"} کا تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("T6", ManualTemplate(
        text='{"placeholder":"text_b"} اور {"placeholder":"text_a"} ایک جیسے خیال رکھتے ہیں؟ {"mask"}',
        tokenizer=tokenizer,
    )),

    ("T7", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے الفاظ کی ہم آہنگی {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("T8", ManualTemplate(
        text='اگر ہم {"placeholder":"text_a"} اور {"placeholder":"text_b"} کو دیکھیں تو یہ {"mask"} ہیں۔',
        tokenizer=tokenizer,
    )),

    ("T9", ManualTemplate(
        text='{"placeholder":"text_a"} کی روشنی میں {"placeholder":"text_b"} {"mask"} سمجھا جا سکتا ہے۔',
        tokenizer=tokenizer,
    )),

    ("T10", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے بیچ مفہوم کا تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),
]


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import os
import re
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from collections import defaultdict
from torch.utils.data import DataLoader, Sampler

# ========================================
# Check CUDA
# ========================================
device = "cuda" #if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


In [3]:
# ==============================
# Balanced Batch Sampler
# ==============================
class BalancedBatchSampler(Sampler):
    def __init__(self, dataset, batch_size):
        """
        dataset: list of InputExample
        batch_size: total batch size (must be divisible by number of classes)
        """
        self.dataset = dataset
        self.labels = [ex.label for ex in dataset]
        self.classes = list(sorted(set(self.labels)))
        self.num_classes = len(self.classes)
        assert batch_size % self.num_classes == 0, "Batch size must be divisible by number of classes"
        self.batch_size_per_class = batch_size // self.num_classes

    def __iter__(self):
        class_indices = {c: np.where(np.array(self.labels) == c)[0].tolist() for c in self.classes}
        for c in self.classes:
            np.random.shuffle(class_indices[c])

        num_batches = min(len(class_indices[c]) // self.batch_size_per_class for c in self.classes)

        for i in range(num_batches):
            batch = []
            for c in self.classes:
                start = i * self.batch_size_per_class
                end = start + self.batch_size_per_class
                batch.extend(class_indices[c][start:end])
            np.random.shuffle(batch)
            yield batch

    def __len__(self):
        return min(len(np.where(np.array(self.labels) == c)[0]) // self.batch_size_per_class for c in self.classes)

In [4]:

# Step 1: Training data (16-shot) ha,m and spam
train_dataset = [

        InputExample(
            guid=0,
            text_a="بینظیر بھٹو پاکستان کی پہلی خاتون وزیراعظم تھیں۔ پہلی بارآپ 1988ء میں پاکستان کی وزیراعظم بنیں لیکن صرف 20 مہینوں کے بعد اس وقت کے صدر پاکستان غلام اسحاق خان نے بے پناہ بدعنوانی کے باعث اپنے خصوصی اختیارت کو استعمال کرتے ہوئے اسمبلی کو ختم کرتے ہوئے نئے الیکشن کروائے۔ بینظیر بھٹو کا تعلق پاکستان پیپلز پارٹی سے تھا۔ ذوالفقار علی بھٹو کی بڑی صاحبزادی سابق وزیراعظم بینظیر بھٹو 21 جون 1953 میں سندھ کے مشہور سیاسی بھٹو خاندان میں پیدا ہوئیں۔ ابتدائی تعلیم کراچی میں حاصل کی Lady Jennings Nursery School اور Convent of Jesus and Mary انھوں نے 15 سال کی عمر بے نظیر بھٹو نے 1973 پولیٹیکل سائنس میں گریجوایشن کر لیا۔ اس کے بعد انھوں نے آکسفورڈ یونیورسٹی میں داخلہ لیا جہاں سے انہوں نے فلسفہ، معاشیات اورسیاسیات میں ایم اے کیا۔ بے نظیر لیاقت باغ میں عوامی جلسے سے خطاب کرنے کے بعد اپنی گاڑی میں بیٹھ کر اسلام آباد آرہی تھیں اس دوران جب وہ پارٹی کارکنوں کے ساتھ اظہار یکجہتی کے لیے گاڑی کی چھت سے باہر نکل رہی تھیں کہ نامعلوم شخص نے ان پر فائرنگ کر دی۔ اس کے بعد بینظیر کی گاڑی سے کچھ فاصلے پر ایک زوردار دھماکہ ہوا اس دھماکے میں بینظیر بھٹو جس گاڑی میں سوار تھیں، اس کو بھی شدید نقصان پہنچا لیکن گاڑی کا ڈرائیور اسی حالت میں گاڑی کو بھگا کر راولپنڈی جنرل ہسپتال لے گیا جہاں وہ زخموں کی تاب نہ لاتے ہوئے جاں بحق ہو گئیں۔",
            text_b="بینظیر بھٹو کو پاکستان کی پہلی خاتون وزیرِ اعظم بننے کا اعزاز حاصل ہے۔ 1988میں پہلی بارآپ نے پاکستان کی وزیرِ اعظم کا عہدہ سنبھالا۔ 20ماہ کی حکومت کے بعد اُس وقت کے صدر غلام اسحاق خان نے بے پناہ بدعنوانیوں کی وجہ سے اپنے اختیارات کا استعمال کیا اور اسمبلی کو تحلیل کر کے نیا الیکشن کرویا۔ بینظیر بھٹو پاکستان پیپلز پارٹی سے تعلق رکھتی تھیں۔ آپ ذوالفقار علی بھٹو کی بیٹی تھیں۔ اورآپ کی پیدائش 21جون 1953میں سندھ کے مشہور سیاسی بھٹو خاندان میں ہوئی۔ بینظیر بھٹو نے اپنی ابتدائی تعلیم کراچی کے جنگ نرسری سکول اور کنوینٹ آف جیسس میری سکول سے حاصل کی۔ 15 سال کی عمر میں او لیولز کا امتحان پاس کرنے کے بعد1973میں گریجوایشن پولیٹیکل سائنسز میں کی۔ اُس کے بعد آکسفورڈ یونیور سٹی سے فلسفہ معاشیات اور سیاسیات میں ایم اے کیا۔ لیاقت باغ راولپنڈی میں جلسہ ِعام سے خطاب کرنے کے بعد آپ اپنی گاڑی میں بیٹھ کر واپس اسلام آباد آرہی تھیں۔ اسی اثناء میں آپ پارٹی کارکنوں سے اظہارِ یکجہتی کے لیے گاڑی کی چھت سے باہر نکلیں تھیں کہ نا معلوم شخص نے آپ پر فائرنگ کر دی۔ پھر کچھ ہی فاصلے پر ایک زوردار دھماکہ ہوا اور آپ کی گاڑی کو بھی شدید نقصان پہنچا۔ ڈرائیور نے گاڑی کوبھگا کر فوری طور پر جنرل ہسپتال راولپنڈی پہنچایا جہاں وہ زخموں کی تاب نہ لاتے ہوئے شہید ہو گئیں۔",
            label=1
        ),
        InputExample(
            guid=1,
            text_a="ملکہ وکٹوریہ سلطنت برطانیہ کی ملکہ تھی۔ ملکہ وکٹوریہ بروز سوموار 24 مئی 1819 ء کو صبح 04:15 پر کینسنگٹن محل لندن میں پیدا ہوئی۔ منگل20 جون1837ء کو 18سال 28 دن کی عمر میں ملکہ برطانیہ بنی اور تا وفات ملکہ رہی۔ 10فروری 1840ء کو شہزادہ البرٹ سے رشتہ ازدواج میں منسلک ہو گئی۔ 14دسمبر 1861ء کو پرنس البرٹ تپ محرقہ کا شکار ہو کر وفات پاگیا۔ اس کے بعد ملکہ نے چالیس سال حالت بیوگی میں اداس تنہا گزارے۔ ملکہ کا کل دور اقتدار 63 سال 7 مہینے 3 دن ہے، اس لحاظ سے وہ سب سے زیادہ دیر تک اقتدار میں رہنے والی برطانوی حکمران تھی۔ بروز منگل 22 جنوری 1901ء کو شام06:30 پر وفات پائی اُس وقت کل عمر 81 سال 8 مہینے تھی۔ سوموار 4 فروری 1901ء کو فراگمور وِنڈسر میں تدفین کی گئی۔ ملکہ وکٹوریہ کے دور میں سلطنت برطانیہ اپنی طاقت اور عالمی اثر و رسوخ میں اپنی انتہا پر تھی اور یہ دور بہت تبدیلیوں کا دور تھا۔ ملکہ کے 9 بچے تھے۔ انگریز دنیا ميں جہاں بھی گئے اپنی اس ملکہ کو نہ بھولے آسٹریلیا کے ایک صوبے کا نام وکٹوریا ہے۔",
            text_b="سلطنت برطا نیہ کی ملکہ جسکا نام ملکہ وکٹوریہ تھا 24مئی 1819ء کو صبح 04:15پیر کے دِن لندن کے ایک عظیم محل کیننگٹن میں پیدا ہوئی۔ ملکہ وکٹوریہ 2دن کم 19 سال کی عمر میں منگل 20جون1837ء کو برطا نیہ کی ملکہ بنی اور اپنی وفا ت تک ملکہ رہیں۔ آپ کی شا دی شہزادہ البرٹ سے 10فروری1840ء کو ہو ئی۔ شہزادہ البرٹ شدید بیماری ٹائیفائیڈ میں مبتلا ہو کر14 دسمبر1861ء کو وفات پانے کے بعدآپ نے اداس اور تنہا 40سال گزارے۔ ملکہ وکٹوریہ کی حکومت 63سال 7مہینے اور 3دن تک رہی اورآپ ہی سب سے زیادہ حکومت میں رہنے والی برطانوی حکمران تھیں۔ آپ نے 81سال 8مہینے کی عمر میں منگل کے دن 22جنوری 1901ء کو شام 06:30اپنے خالقِ حقیقی سے جا ملیں آپ گوفراگموروِنڈسر میں 4فروری1901ء کو دفن کیا گیا۔ سلطنت برطانیہ کے ملکہ وکٹوریہ کے حکومت اور نظامی عمل عروج کی بلندوں پر تھی۔ اس دورِ حکومت میں بہت سی تبدیلیاں آئیں۔ ملکہ وکٹوریہ کے 9بچے تھے۔ ملکہ کو اس برطانوی حکمرانی میں بہت عزتوشہرت ملی۔ انگریزدنیا کے جس بھی کونے میں چلے جاتے ملکہ کو نہیں بھولتے تھے۔ آسٹریلیا کا ایک صوبہ جس کا نام وکٹوریہ ہے ملکہ کی یاد میں رکھا گیا۔",
            label=1
        ),
        InputExample(
            guid=2,
            text_a="شیرشاہ سوری کا اصل نام فرید خان تھا۔ 1486ع میں پیدا ہوئے جونپور میں تعلیم پائی۔ 21 سال والد کی جاگیر کا انتظام چلایا پھر والئ بہار کی ملازمت کی۔ جنوبی بہار کے گورنر بنے۔ کچھ عرصہ شہنشاہ بابر کی ملازمت کی بنگال بہاراور قنوج پر قبضہ کیا مغلشہنشاہ ہمایوں کو شکست دے کر ہندوستان پر اپنی حکمرانی قائم کی۔ اپنی مملکت میں بہت سی اصلاحات نافذ کیں۔ اپنے تعمیری کاموں کی وجہ سے ہندوستان کے نپولین کہلائے سنار گاؤں سے دریائے سندھ تک ایک ہزار پانچ سو کوس لمبی جرنیلی سڑک تعمیر کروائی جو آج تک جی ٹی روڈ کے نام سے موجود ہے۔ تاریخ دان شیر شاہ سوری کو برصغیر کی اسلامی تاریخ کا عظیم رہنما, فاتح اور مصلح مانتے ہیں۔ اردو ادب میں شیر شاہ سوری سے متعلق کئی مثالی قصے ملتے ہیں۔ اپنی جدوجہد سے شیر شاہ نے عظیم سلطنت قائم کی تھی۔ لیکن اس نے اپنی آخری آرام گاہ کے طور پر سہسرام کو ہی پسند کیا تھا اس لیے اپنی زندگی میں ہی کیمور کی پہاڑی پر مقبرہ تعمیر کروایا تھا جس کے چاروں جانب جھیل پھیلی ہوئی ہے۔",
            text_b="شیر شاہ سوری کی تاریخ پیدائش 1486ء ہے آپ کا اصل نام فر ید خان تھا۔ آپ نے ابتدائی تعلیم جورنپور سے حاصل کی۔ آپ نے اپنے باپ کی جائیدات کے نظام کو 21سال باخوبی طریقے سے چلانے کے بعد کھیتی باڈی کی ملازمت کی آپ جنوبی کھیتی باڈی کی گورنر بنے۔ آپ نے ہندوستان پر اپنی حکمرانی مغلشہنشاہ ہمایوں کو شکست دے کر قائم کی کچھ دیر شہنشاہ بابر کی نوکری کر کے قنوج اور بنگال بہار پر قبضہ کر لیا۔ آپ نے بہت سے تعمیرِی عمل قائم کیے۔ اور اپنی حکمرانی میں ہندوستان کے پنولین اسی تعمیرِی کاموں کی بدولت کہلائے۔ شر شاہ سوری نے ایک بہت لمبی جرنیلی سٹرک جسے آج بھی جی ٹی روڈ کے نام سے جانا جاتا ہے تعمیر کروائی جس کی لمبائی سنار گاؤں سے لے کر دریائے سندھ تک ایک ہزار پانچ سو میلوں ہے۔ شیر شاہ سوری کو تاریخ میں بہت اہمیت حاصل ہے۔ پاک وہند کی اسلامی تاریخ کا فلاحیِ کام کرنے والا،اعظم رہنما ،اور اول کندہ تاریخ دان انھیں کو مانتے ہیں۔ اِن کے بارے میں بہت سے مثالی قصے اردوادب میں بھی ملتے ہیں۔ آپ نے شدید محنت سے وعظیم سلطنت بنانے کے باوجود اپنی آخری سکون کی جگہ سہسرام کو ہی چنا جہاں آپ نے اپنی زندگی میں ہی کمیور کی پہاڈی جس کے چاروں طرف جھیل بہتی ہے پر اپنا مقبرہ تعمیرکروایا تھا۔",
            label=1
        ),
        InputExample(
            guid=3,
            text_a="پاکستانی سائسندان پاکستانی ایٹم بم کے خالق ہندوستان کے شہر بھوپال میں پیدا ہوئے ۔ ڈاکٹر قدیرخان پندرہ برس یورپ میں رہنے کے دوران ہالینڈ سے ماسٹرز آف سائنس جبکہ بیلجیئم سے ڈاکٹریٹ آف انجینئرنگ کی اسناد حاصل کرنے کے بعد 31 مئی 1976ء میں انہوں نے ذوالفقار علی بھٹو سے مل کر انجینئرنگ ریسرچ لیبارٹریزمیں شمولیت اختیار کی ۔ اس ادارے کا نام یکم مئی 1981ء کو جنرل ضیاء الحق نے تبدیل کر کے ’ ڈاکٹر اے کیو خان ریسرچ لیبارٹریز ‘ رکھ دیا ۔ڈاکٹر قدیر خان پر ہالینڈ کی حکومت نے اہم معلومات چرانے کے الزامات کے تحت مقدمہ بھی دائر کیا جن معلومات کو چرانے کی بنا پر مقدمہ داخل کیا گیا ہے وہ عام اور کتابوں میں موجود ہیںـ جس کے بعد ہالینڈ کی عدالت عالیہ نے ان کو باعزت بری کر دیا تھا ۔ مئی 1998ء میں پاکستان نے بھارتی ایٹم بم کے تجربے کے بعد کامیاب تجربہ کیا ۔ ـڈاکٹر قدیر خان کو وقت بوقت 13 طلائی تمغے ملے ۔ ڈاکٹر قدیرخان نے ہالینڈ میں قیام کے دوران ایک مقامی لڑکی ہنی خان سے شادی کی اور جن سے ان کی دو بیٹیاں ہوئیں ۔2004 ان پر یٹمی آلات دوسرے اسلامی ممالک کو فروخت کرنے کا الزام لگا ۔ جس کے جواب میں انہوں نے یہ ناقابل یقین اعتراف کیا کہ انہوں نے ایسا کیا ہے اور حکومت پاکستان کا یا فوج کا اس میں کوئی ہاتھ نہیں رہا ۔ اس کے بعد قدیر خان کو نظر بند کر دیا گیا ۔ اگست 2006ء میں انکشاف ہوا کہ قدیر خان کو کینسر ہے ۔",
            text_b="پاکستان کو ایٹمی طاقت بنانے والے عبد القدیر خان اور پاکستانی سائنسدان کی پیدائش ہندوستان کے شہر بھوپال میں ہوئی۔ آپ نے اپنی زندگی کے پندرہ برس یورپ میں گزارے اور اسی دوران آپ نے ماسٹرز آف سائنس ہالینڈ سے کیا۔ ماسٹرز کے بعد ڈاکٹریٹ آف انجیئنرنگ کی اسناد بیلجیئم سے حاصل کیں۔ تعلیم مکمل کرنے کے بعد ذوالفقار علی بھٹو کے ساتھ مل کر 31مئی 1976ء میں انجیئنرنگ ریسرچ لیبارٹریز میں شامل ہوئے۔ جنرل ضیاءالحق نے اس ادارے کا نام یکم مئی 1981ء کو 'ڈاکٹر اے کیو خان ریسرچ لیبارٹریز ' رکھ دیا۔ ہالینڈ کی حکومت نے ڈاکٹر قدیر خان پر اہم معلومات کو چرانے کا مقدمہ بھی درج کیا۔ جن معلومات کہ چرانے کا الزام آپ پر لگایا گیا وہ وہاں کی عام کتابوں میں بھی موجود ہیں۔ ہالینڈ کی اعلی عدلیہ نے آپ پر الزام غلط ثابت ہونے پرآپ کو باعزت بری کر دیا۔ پاکستان نے اپنے پڑوسی بھارت کے ایٹم بم کے تجربے کے مقابلے میں مئی 998ء میں کامیاب تجربہ کیا۔ ڈاکٹر قدیر خان نے اپنی کامیابیوں پر وقت بوقت 13طلائی طمغے حاصل کئے۔ہالینڈ میں قیام کے دوران آپ نے وہاں کی ایک مقامی لڑکی جس کا نام ہنی خان ہے سے شادی کی۔ اور آپ دو بیٹیوں کے باپ بنے۔ آپ پر ایٹمی ٹیکنالوجی دوسرے اسلامی ممالک کو بیچنے کا الزام 2004میں لگا۔ اس الزام کے جواب انہوں نے اعتراف کر کے حیران کر دیا کہ وہ ایسا کر چکےہیں۔ انہوں نے یہ بھی کہا کہ حکومت ِ پاکستان اور پاکستانی فوج اِس میں ہر گز ملوث نہیں ہے۔ اس اعتراف کے بعد سے آپ کو نظر بندی کاسامنا کرنا پڑا۔ عبد القدیر خان کو کینسر کی بیماری کا انکشاف 2006ء میں ہوا۔",
            label=1
        ),
        InputExample(
            guid=4,
            text_a="بلھے شاہ کا اصل نام عبد اللہ شاہ تھا۔ 1680ء میں مغلیہ سلطنت کے عروج میں اوچ گیلانیاں میں پیدا ہوئے کچھ عرصہ یہاں رہنے کے بعد قصور کے قریب پانڈو میں منتقل ہو گئے۔ ابتدائی تعلیم یہیں حاصل کی۔ ان کا تعلق صوفیاء کے قادریہ مکتبہ فکرسے تھا۔ ان کی ذہنی نشوونما میں قادریہ کے علاوہ شطاریہ فکر نے بھی نمایاں کردار ادا کیا تھا۔ اسی لئے ان کی شاعری کے باغیانہ فکر کی بعض بنیادی خصوصیات شطاریوں سے مستعار ہیں۔ صوفیانہ مسائل پر گہری نظر رکھتے تھے اور قادریہ سلسلے سے بھی بیعت تھے اس لئے ان کی ذات میں یہ دونوں سلسلے مل کر ایک نئی ترکیب کا موجب بنے۔ بلھے شاہ انہی شاہ عنایت کے مرید تھے۔ بلھے شاہ کی شاعری عالمگیری عقیدہ پرستی کے خلاف رد عمل ہے۔ ان کی شاعری میں صلح کل، انسان دوستی، اور عالم گیر محبت کا جود رس ملتا ہے بلھے شاہ کا انتقال 1757ء میں قصور میں ہوا اور یہیں دفن ہوئے۔ ان کے مزار پر آج تک عقیدت مند ہر سال ان کی صوفیانہ شاعری گا کر انہیں خراج عقیدت پیش کرتے ہیں۔",
            text_b="مغلیہ سلطنت کی سربلندوں میں اوچ گیلانیاں میں 1680ء میں پیدا ہوئے۔ آپ کا اصل نام عبداللہ شاہ تھا۔ کچھ دیر اوچ گیلانیاں میں رہنے کے بعد پانڈو جو کہ قصور کے قریب ہے وہاں رِہائش پذیر ہوگئے۔ اِبتدائی تعلیم بھی یہیں سے حاصل کی بلھے شاہ کا تعلق صوفیاِء اکرام کے قادریہ علمی سوچ سے تھا۔ قادریہ کی باطنیِ علمی سوچ کے ساتھ ساتھ روحانی سوچ نے بھی آپ کی ذِہنی نشونما میں اِہم کردار ادا کیا تھا۔ بلھے شاہ کی شاعری کی فسادی سوچ کی بہت سی اہم خصو صیات اِنکی ر وحانیت سے مجاذی ہیں۔ آپ اسلامی مسائل پر گہری نظر رکھتے اور ساتھ ساتھ قادریہ سلسلے سے بھی منسلک رہتے تھے۔ اس لیے یہ دونوں سلسلے مل کر آپ کی ذات میں ایک نئی ایجاد کا ذریعہ ہے۔ بلھے شاہ عنایت کے مرید تھے اِن کی شاعری میں اِنسانی دوستی ،تحریک اتحاد اور عالم گیر محبت کا پرْزور جذبہ کا رنگ نظر آتا ہے ۔اِن کی شاعری ایمان پرستی والوں کے خلاف ایک خاص ردِعمل ہے۔ بلھے شاہ نے 1757ء کو قصور میں وفات پائی آپ کو یہیں دفن بھی کیا گیا۔ آج بھی بہت سے عقیدت مند ہرسال اِن کے مزار پر تشریف آور ہوتے اور اِن کی خاص صوفیانہ شاعری گاکر خراجِ وعقیدت پیش کرتے ہیں۔",
            label=1
        ),
        InputExample(
            guid=5,
            text_a="اعزازِ نشان حیدر حاصل کرنے والے پائلٹ آفیسر کراچی میں پیدا ہوئے۔ منہاس راجپوت گوت گھرانے کے چشم چراغ تھے۔ 1968ء میں سینٹ پیٹرک سکول کراچی سے سینئر کیمبرج کیا۔ خاندان کے متعدد افراد پاکستان کی بری بحری اور فضائی افواج میں اعلیٰ عہدوں پر فائز تھے۔ فروری 1971ء میں پشاور یونیورسٹی سے انگریزی، ائیر فورس لا ملٹری ہسٹری الیکڑونکس موسمیات جہازرانی ہوائی حرکیات وغیرہ میں بی۔ ایس۔ سی کیا۔ اور اگست 1971ء میں پائلٹ آفیسر بنے۔20 اگست 1971ء کو راشد کی تیسری تنہا پرواز تھی۔ وہ ٹرینر جیٹ طیارے میں سوار ہوئے ہی تھے کہ ان کا انسٹرکٹر سیفٹی فلائٹ آفیسر مطیع الرحمان خطرے کا سگنل دے کر کاک پٹ میں داخل ہوگیا اور طیارے کا رخ بھارت کی سرحد کی طرف موڑ دیا۔ اگلے پانچ منٹ راشد اور انسٹرکٹر کے درمیان کشمکش میں گزرے اور اسی کش مکش کے دوران طیارہ زمین پر گر کر تباہ ہوگیا۔ راشد نے شہادت کا درجہ پایا اور انھیں اس عظیم کارنامے کے صلے میں سب سے بڑا فوجی اعزاز نشانِ حیدر دیا گیا۔",
            text_b="پائلٹ آ فسر راشد منہاس جن کو امتیازی نشان 'نشانِ حیدر' ملا کراچی میں پیدا ہوئے۔ راشد منہاس راجپوت جوتے بنانے والے گھرانے کے اعظم انسان تھے۔ سینٹر کیمبرج جوکہ 1968 ء میں آپ نے سینٹ میٹرک سکول کراچی سے کیا۔ پاکستان کی بری بحری اور فضائی افواج میں اس خاندان کے کثیرافراد ہی اعلیٰ درجے پر تخت نشن تھے راشد منہاس نے پشاور یو نیورسٹی سے انگریزی،ائیرفورس،ملٹری سینٹری،الیکٹرونکس،موسمیات،جہازرانی،ہوائی حرکیات وغیرہ میں 1971ء میں بی۔ ایس۔سی۔کیا۔ اورآخر کار آپ اگست1971ء میں پائلٹ آفسر بنے۔ راشد منہاس کی تیسری اکیلے فلائیٹ انجنئیر20 اگست 1971ء کو تھی۔ وہ ٹر ینر جیٹ طیارے کے خول میں سوار ہوئے ہی تھے کہ آپ کا ساتھی مطیع الرحمان انسٹر کیٹر سیفٹی فلائٹ آفسر نے خطرے کی نشاندہی کرتے ہوئے طیارے کو بھارت کی حدود کی طرف موڑتے ہوئے کاک پٹ میں داخل ہو گیا۔ اور اگلا وقت راشد منہاس اور انسٹر کٹر کے درمیان مسلسل خیالات میں گم گزررہا تھا کہ اسی دوران طیارہ زمین پر جسں کی وجہ سے راشد منہاس کو شہادت نوش ہوئی اور آپ کو بہت بٹرا اور فوجی اعزاز نشانِ حیدر اِسی اعظم کارنامے کی بدولت ملا۔",
            label=1
        ),
        InputExample(
            guid=6,
            text_a="بینظیر بھٹو پاکستان کی پہلی خاتون وزیراعظم تھیں۔ پہلی بارآپ 1988ء میں پاکستان کی وزیراعظم بنیں لیکن صرف 20 مہینوں کے بعد اس وقت کے صدر پاکستان غلام اسحاق خان نے بے پناہ بدعنوانی کے باعث اپنے خصوصی اختیارت کو استعمال کرتے ہوئے اسمبلی کو ختم کرتے ہوئے نئے الیکشن کروائے۔ بینظیر بھٹو کا تعلق پاکستان پیپلز پارٹی سے تھا۔ ذوالفقار علی بھٹو کی بڑی صاحبزادی سابق وزیراعظم بینظیر بھٹو 21 جون 1953 میں سندھ کے مشہور سیاسی بھٹو خاندان میں پیدا ہوئیں۔ ابتدائی تعلیم کراچی میں حاصل کی Lady Jennings Nursery School اور Convent of Jesus and Mary انھوں نے 15 سال کی عمر بے نظیر بھٹو نے 1973 پولیٹیکل سائنس میں گریجوایشن کر لیا۔ اس کے بعد انھوں نے آکسفورڈ یونیورسٹی میں داخلہ لیا جہاں سے انہوں نے فلسفہ، معاشیات اورسیاسیات میں ایم اے کیا۔ بے نظیر لیاقت باغ میں عوامی جلسے سے خطاب کرنے کے بعد اپنی گاڑی میں بیٹھ کر اسلام آباد آرہی تھیں اس دوران جب وہ پارٹی کارکنوں کے ساتھ اظہار یکجہتی کے لیے گاڑی کی چھت سے باہر نکل رہی تھیں کہ نامعلوم شخص نے ان پر فائرنگ کر دی۔ اس کے بعد بینظیر کی گاڑی سے کچھ فاصلے پر ایک زوردار دھماکہ ہوا اس دھماکے میں بینظیر بھٹو جس گاڑی میں سوار تھیں، اس کو بھی شدید نقصان پہنچا لیکن گاڑی کا ڈرائیور اسی حالت میں گاڑی کو بھگا کر راولپنڈی جنرل ہسپتال لے گیا جہاں وہ زخموں کی تاب نہ لاتے ہوئے جاں بحق ہو گئیں۔",
            text_b="پاکستان کی پہلی وزیراعظم خاتون بینظیربھٹو تھیں۔ 1988ء میں بینظیر بھٹوپہلی بار پاکستان کی زیراعظم بنیں۔ لیکن اس وقت کے صدر غلام اسیق خان نے اپنے حاصوصی اختیارات استعمال کر تے ہوئے بے پناہ بد انتظامی کے سبب اسملی کو ختم کرتے ہو ئے صرف 20مہینوں کے بعد ہی نئے الیکشن کروائے۔ بینظیر بھٹو کا واسطہ پاکستان پیپلزپارٹی سے تھا۔ 21جون 1953ء میں سابق وازیراعظم ذوالفقارعلی بھٹو کی بٹری بیٹی بینظیر بھٹو سندھ کے سیاسی بھٹو کنبے میں پیدا ہو ئیں۔Lady Jenning Nursery SchoolاورConvent of Jesus and Maryکراچی سے بینظیربھٹو نے ابتدائی تعلیم حاصل کی۔ اولیول کا امتحان انہوں نے15سال کی عمر میں انہوں نے پاس کیا۔ پولیٹیکل سائنس میں1973ء بینظیر بھٹونے گرایجوایشن کر لیا۔ اس کے بعد انہوں نے آکسفورڑ یونیورسٹی میں داخلہ لیا۔ جہاں سے انہوں نے سیاست ‘ فلسفہ‘ معاشیات میں ایم اے کیا۔ بینظیر بھٹو عوامی جلسے سے لیاقت باغ میں خظاب کرنے کے بعد اسلام آباد اپنی گاڑی میں بیٹھ کرآرہی تھیں۔ اس دوران پیپلز پارٹی کے کارکنوں کے سا تھ اظہارِ یکجہتی کے لئے گاڑی کی چھت سے باہر نکل رہی تھیں۔ کہ انجان شخص نے ان پر فائرنگ کر دی۔ اس کے بعد پرزور دھماکہ بینظیر بھٹو کی گاڑی سے کچھ فاصلہ پر ہوا۔ اس دھماکہ سے بینظیر بھٹو جس گاڑی میں سوار تھیں۔ اس کو بہت نقصان پہنچا۔ لیکن ڑرائیور گاڑی کو بھگا کرروالپنڈی کے جنرل ہسپتال لے گیا۔ جہاں وہ ذخموں کو برداشت نہ کرتے ہوئے مر گئیں۔",
            label=1
        ),
        InputExample(
            guid=7,
            text_a="علامہ اقبال 9 نومبر۱۸۷۷ء بمطابق 3 ذیقعد 1294ھ کو برطانوی ہندوستان کے شہر سیال کوٹ میں شیخ نور محمد کے گھر پیدا ہوئے۔ ماں باپ نے نام محمد اقبال رکھا۔ ۔شیخ نُور محمد دیندار آدمی تھے سیالکوٹ کے اکثر مقامی علماء کے ساتھ دوستانہ مراسم تھے۔ اقبال بسم اﷲ کی عمر کو پہنچے تو انھیں مولانا غلام حسن کے پاس لے گئے۔ یہاں سے اقبال کی تعلیم کا آغاز ہوا۔ حسبِ دستور قرآن شریف سے ابتداء ہوئی۔ تقریباً سال بھر تک یہ سلسلہ چلتا رہا کہ ایک شہر کے نامور عالم مولانا سید میر حسن ادھر آ نکلے۔ ایک بچّے کو بیٹھے دیکھا کہ صورت سے عظمت اور سعادت کی پہلی جوت چمکتی نظر آرہی تھی۔ پوچھا کہ کس کا بچّہ ہے۔ معلوم ہوا تو وہاں سے اُٹھ کر شیخ نور محمد کی طرف چل پڑے ۔دونوں آپس میں قریبی واقف تھے۔ مولانا نے زور دے کر سمجھایا کہ اپنے بیٹے کو مدرسے تک محدود نہ رکھو۔ اس کے لیے جدید تعلیم بھی بہت ضروری ہے انھوں نے خواہش ظاہر کی کہ اقبال کو ان کی تربیت میں دے دیا جائے۔ کچھ دن تک تو شیخ نور محمد کو پس و پیش رہا، مگر جب دوسری طرف سے اصرار بڑھتا چلا گیا تو اقبال کو میر حسن کے سپرد کردیا یہاں اقبال نے اردو، فارسی اور عربی ادب پڑھنا شروع کیا۔ تین سال گزر گئے۔ اسکول سے آتے تو استاد کی خدمت میں پہنچ جاتے۔ میر حسن ان اچھے زمانے میں استاد مرشد ہوا کرتا تھا۔",
            text_b="برطانوی ہندوستان میں رہنے والے شیخ محمد نور کے گھر علامہ اقبال 9 نومبر 1877 بمطابق 3ذیقعد 1294ءکو سیالکوٹ میں پیدا ہوئے۔ والدین نے محمد اقبال نام رکھا۔ اقبال کے والد محترم دیندار انسان تھے۔ اپنے شہر کے کئی علماءکے ساتھ نور محمد کے دوستانہ تعلقات تھے۔ اقبال کی تعلیم کا آغاز مولاناغلام حسن کے ہاں سے بسم اللہ کی عمر سے ہوا۔ رواج کے مطابق قرآن مجید سے آغازہوا۔ سارا سالقرآن کی تعلیم لی کہ اِسی دوران ایک مشہور عالم مولانا سید میر حسن سیالکوٹ آئے۔ بچے کو بیٹھے دیکھا تو چہرے سے سعادت اور عظمت کی روشنی چمکتی دکھائی دی۔ بچے کے سر پرستوں کا پوچھا پتہ چلنے پر علامہ اقبال کے والد کی طرف چل پڑے جن کے آپ قریبی واقف تھے۔ مولانا نے سمجھاتے ہوئےکہاکہ اقبال کو مدرسے کی تعلیم تک محدود نہ رکھو جدید تعلیم سے آراستگی بہت ضروری ہے۔ اپنی تعلیم و اخلاق میں اقبال کو رکھنے کی مولانا نے خواہش ظاہر کی۔ نور محمد کچھ دن سوچ بچار کرتے رہے۔ مولانا کی کافی ضد کےبعد آپ نے اقبال کو اِ ن کی نگرانی میں دے دیا۔ فارسی ، اردو اور عربی آپ نے وہاں پڑھنا شروع کیا۔ تین سال بیت گئے۔ اسکول سے واپسی کے بعد استاد کی خدمت میں پہنچ جاتے۔ استاد اُس زمانے میں مرشد ہوا کرتا تھا۔",
            label=1
        ),
        InputExample(
            guid=8,
            text_a="اعزازِ نشان حیدر حاصل کرنے والے پائلٹ آفیسر کراچی میں پیدا ہوئے۔ منہاس راجپوت گوت گھرانے کے چشم چراغ تھے۔ 1968ء میں سینٹ پیٹرک سکول کراچی سے سینئر کیمبرج کیا۔ خاندان کے متعدد افراد پاکستان کی بری بحری اور فضائی افواج میں اعلیٰ عہدوں پر فائز تھے۔ فروری 1971ء میں پشاور یونیورسٹی سے انگریزی، ائیر فورس لا ملٹری ہسٹری الیکڑونکس موسمیات جہازرانی ہوائی حرکیات وغیرہ میں بی۔ ایس۔ سی کیا۔ اور اگست 1971ء میں پائلٹ آفیسر بنے۔20 اگست 1971ء کو راشد کی تیسری تنہا پرواز تھی۔ وہ ٹرینر جیٹ طیارے میں سوار ہوئے ہی تھے کہ ان کا انسٹرکٹر سیفٹی فلائٹ آفیسر مطیع الرحمان خطرے کا سگنل دے کر کاک پٹ میں داخل ہوگیا اور طیارے کا رخ بھارت کی سرحد کی طرف موڑ دیا۔ اگلے پانچ منٹ راشد اور انسٹرکٹر کے درمیان کشمکش میں گزرے اور اسی کش مکش کے دوران طیارہ زمین پر گر کر تباہ ہوگیا۔ راشد نے شہادت کا درجہ پایا اور انھیں اس عظیم کارنامے کے صلے میں سب سے بڑا فوجی اعزاز نشانِ حیدر دیا گیا۔",
            text_b="کراچی میں پیدا ہونے والے پائلٹ آفیسر راشد منہاس کو نشانِ حیدر سے نوازا گیا۔ آپ نے منہاس راجپوت گوت گھرانے میں آنکھ کھولی۔ آپ نے کراچی کے ہی سینٹ پیٹرک سکول سے سینئر کیمبرج کیا۔ آپ کے گھرانے کے کئی لوگ پاکستان کی بحری ، بری اور فضائی افواج میں کئی اعلیٰ عہدوں پر اپنی خدمات انجام دے رہے تھے۔ راشد منہاس نے انگریزی، ایئر فورس لاء، ملٹری ہسٹری ، الیکٹرونکس ، موسمیات جہاز رانی ، ہوائی حرکات وغیرہ میں بی۔ایس۔سی،کی تعلیم فروری 1971ء میں پشاور یونیورسٹی سے لی۔ اور اسی سال پائلٹ آفیسر بن گئے۔ تیسری تنہا پرواز آپ نےبیس اگست 1971ء کو کی۔ راشد منہاس ابھی ٹرینر جیٹ میں بیٹھے ہی تھےانہیں اپنے انسٹرکٹر سے خطرہ محسوس ہوا۔ سیفٹی فلائٹ آفیسر مطیع الرحمان خطرے کا سگنل دے کر کاک پٹ میں آگیا۔ اور جیٹ کو بھارت کی سرحد کی طرف لے گیا۔ 5منٹ تک راشد منہاس اور انسٹرکٹر کے درمیان کھینچا تانی ہوتی رہی اور اسی دوران طیارہ زمین سے ٹکرا کر تباہ ہو گیا۔ راشد منہاس شہادت کے رتبہ پر فائز ہوئےاور اسی عظیم قربانی کے انعام میں انہیں نشانِ حیدر سے نوازا گیا۔ جو کہ پاکستان فوج کا سب سے بڑا اعزاز ہے۔",
            label=1
        ),
        InputExample(
            guid=9,
            text_a="حضرت زینب سلام اللہ علیہا امام علی علیہ السلام اور حضرت فاطمہ سلام اللہ علیہا کی بیٹی یعنی حضرت محمد صلی اللہ علیہ و آلہ و سلم کی نواسی تھیں۔ وہ 5 جمادی الاول 6ھ کو مدینہ میں پیدا ہوئیں۔ واقعہ کربلا کے بعد حضرت زینب سلام اللہ علیہا کا کردار بہت اہم ہے۔ واقعہ کربلا کے بعد وہ دمشق لے جائی گئیں جہاں یزید کے دربار میں دیا گیا ان کا خطبہ بہت مشہور ہے۔ آپ نے اپنے خطبے میں فرمایا۔ یزيد اگر چہ حادثات زمانہ نے ہمیں اس موڑ پر لا کھڑا کیا ہے اور مجھے قیدی بنایا گیا ہے لیکن جان لے میرے نزدیک تیری طاقت کچھ بھی نہیں ہے۔ خدا کی قسم، خدا کے سوا کسی سے نہیں ڈرتی اس کے سوا کسی اور سے گلہ و شکوہ بھی نہیں کروں گی۔ ہم اہل بیت پیغمبر سے دشمنی کے لئے تو جتنی بھی سازشیں کرسکتا ہے کر لے لیکن خدا کی قسم تو ہمارے نام کو لوگوں کے دل و ذہن اور تاریخ سے نہیں مٹا سکتا اور اسی طرح تو اپنے دامن پر لگے ننگ و عارکے بدنما داغ کو بھی نہیں دھو سکتا، خدا کی نفرین و لعنت ہوظالموں اور ستمگروں پر۔ حضرت زینب اپنے بھائی امام حسین کی شہادت کے تقریبا' ڈیڑھ سال بعد 15 رجب المرجب 62 ہجری کودرجہ شہادت پر فائز ہوئیں۔ ان کا روضہ اقدس شام کے دارالحکومت دمشق میں ہے۔",
            text_b="حضرت محمد ؐکی نواسی یعنی حضرت فاطمہؑ اور امام علی علیہ السلام کی بیٹی حضرت زینبؑ ہیں۔ آپؑ مدینہ میں 5 جمادی الاول 6ھ کو پیدا ہوئیں۔ حضرت زینبؑ کا کردار کربلا کے واقعے کے بعد بہت اہم ہے۔ جس کے بعد آپؑ کو دمشق لے جایا گیا اور یزید کے دربار میں پیش کیا گیا جہاں آپؑ نے خطبہ دیا جو بہت مشہور ہے۔ خطبہ میں آپؑ نے فرمایا۔ زمانے کے حادثوں نے مجھے اِس موڑ پر لا کھڑا کیا ہے کہ میں قیدی بنا دی گئی ہوں۔ لیکن یزید میرے قریب تیری طاقت کچھ بھی نہیں ہے۔ خُدا کی قسم اُس کے سوا نا کسی سے ڈرتی ہوں اور نہ ہی اُس کے علاوہ کسی سے گلہ شکوہ بھی کروں گی۔ یزید تو ہم اہلِ بیت پیغمبرؐ سے جتنی بھی دشمنی میں سازشیں کر سکتا ہے کر لے۔ لیکن خداکی قسم تو تاریخ اور لوگوں کے دل و زہن سے ہمیں ختم نہیں کر سکتا۔ تو اپنے سینے پر لگے ننگ و عار کے بد نمانشانوں کو بھی نہیں صاف کر سکتا۔ خدا کی لعنت و ملامت ہو ستمگروں اور ظلم کرنے والوں پرحضرت زینبؑ کو شہادت کا مرتبہ اپنے بھائی امام حسین سے شہادت کے تقریبا ڈیڑھ سال بعد 15 رجب المرجب 62 ہجری کو ملا۔ شام کے دارالحکومت دمشق میں آپ کا روضہ اقدس ہے۔",
            label=1
        ),
        InputExample(
            guid=10,
            text_a="عظیم پاکستانی قوال، موسیقار اور گلوکار فیصل آباد میں پیدا ہوئے۔ آپ کے والد فتح علی خان اور تایا مبارک علی خان اپنے وقت کے مشہور قوال تھے۔ ان کے خاندان نے قیام پاکستان کے وقت مشرقی پنجاب کی ضلع جالندھر سے ہجرت کر کے فیصل آباد میں سکونت اختیار کی تھی۔ استاد نصرت فتح علی خان نے اپنی تمام عمر قوالی کے فن کو سیکھنے اور اسے مشرق و مغرب میں مقبول عام بنانے میں صرف کر دی۔ انہوں نے صوفیائے کرام کے پیغام کو دنیا کے کونے کونے تک پہنچایا اور ان کے فیض سے خود انہیں بے پناہ شہرت نصیب ہوئی۔پاکستان میں نصرت فتح علی خان کا پہلا تعارف اپنے خاندان کی روایتی رنگ میں گائی ہوئی ان کی ابتدائی قوالیوں سے ہوا۔ ان کی مشہور قوالی ’علی مولا علی‘ انہں دنوں کی یادگار ہے۔ بعد میں انہوں نے لوک شاعری اور اپنے ہم عصر شعراء کا کلام اپنے مخصوص انداز میں گا کر ملک کے اندر کامیابی کے جھنڈے گاڑے عام طور پر قوالی سے لگاؤ نہ رکھنے والے طبقے کو اپنی طرف راغب کیا اور یوں نصرت فتح علی خان کا حلقہ اثر وسیع تر ہو گیا۔",
            text_b="پاکستانی عظیم گانے والے موسیقار اور قوال نصرت فتح علی فیصل ٓاباد میں پیدا ھوے۔ آپ کے والد محترم فتح علی خان اور چچا مبارک علی خان اپنے وقت کے بہت مشہور عزت کے حامل قوال تھے۔ قیامِ پاکستان کے وقت آپ کے خاندان نے مشرقی پنجاب کے ضلع جالندھر سے فیصل آباد میں رہائش پذیر ہوئے۔ استاد نصرت فتح علی خان نے اپنی ساری زندگی قوالی کے فن کو سیکھنے اور اِس کو مشرق و مغرب میں ہر دل عزیز بنانے میں وقف کر دی۔ نصرت فتح علی خان کو اپنے صوفیائے کرام کی رسائی سے بہت زیادہ عزت ملی۔ اور انھوں نے ان کے پیغام کو دنیا کے ہر حصے تک پہنچا یا۔ نصرت فتح علی خان کا پاکستان میں اپنی موسیقی کا آغاز اپنے خاندان کے روایتی رنگ میں گائے جانے والی ابتدائی قوالیوں سے ہوا۔ اِن کی مشہور قوالی 'علی مولا علی'اپنے ہم عہد شعراء کا لوک شاعری ان دنوں کی ایک عظیم یادگار ہے۔ بعد میں کلام کو اپنے خاص انداز میں گا کر ملک کے اندر کامیابی کو سر بلند کیا۔ قوالی سے دلچسپی نہ رکھنے والے طبقہ کو نصرت فتح علی خان نے اپنی طرف متوجہ کیا۔ اور اس طرح آپ کا مباحثہ اثروسیع تر ہو گیا۔",
            label=1
        ),
        InputExample(
            guid=11,
            text_a="پاکستان کے پہلے منتخب وزیر اعظم لاڑکانہ سندھ میں پیدا ہوئے۔ انکے والد سر شاہ نواز بھٹو مشیر اعلی حکومت بمبئی اور جونا گڑھ کی مسلم ریاست میں دیوان تھے۔ ذوالفقار علی بھٹو نے 1950 میں برکلے یونیورسٹی کیلیفورنیا سے سیاسیات میں گریجویشن کی۔ 1952ء میں آکسفورڈ یونیورسٹی سے اصول قانون میں ماسٹر کی ڈگری لی۔ اسی سال مڈل ٹمپل لندن سے بیرسٹری کا امتحان پاس کیا۔ پہلے ایشیائی تھے جنھیں انگلستان کی ایک یونیورسٹی ’’ساؤ تھمپئین‘‘ میں بین الاقوامی قانون کا استاد مقرر کیا گیا۔ کچھ عرصہ مسلم لاء کالج کراچی میں دستوری قانون کے لیکچرر رہے۔ 1953ء میں سندھ ہائی کورٹ میں وکالت شروع کی۔ دسمبر 1967ء میں پاکستان پیپلز پارٹی کی بنیاد رکھی۔ 1970ء کے عام انتخابات میں پیپلز پارٹی نے مغربی پاکستان میں نمایاں کامیابی حاصل کی۔ دسمبر 1971ء میں جنرل یحیٰی خان نے پاکستان کی عنان حکومت مسٹر بھٹو کو سونپ دی۔ وہ دسمبر 1971ء تا 13 اگست 1973 صدر مملکت کے عہدے پر فائز رہے۔ 14 اگست 1973ء کو نئے آئین کے تحت وزیراعظم کا حلف اٹھایا۔ 1977ء کے عام انتخابات میں دھاندلیوں کے سبب ملک میں خانہ جنگی کی سی کیفیت پیدا ہو گئی۔ 5 جولائی 1977 کو جنرل محمد ضیا الحق نے مارشل لاء نافذ کر دیا۔ ستمبر 1977ء میں مسٹر بھٹو نواب محمد احمد خاں کے قتل کے الزام میں گرفتار کر لیے گئے۔ 18 مارچ 1978ء کو ہائی کورٹ نے انھیں سزائے موت کا حکم سنایا۔ 6 فروری 1979ء کو سپریم کورٹ نے ہائی کورٹ کے فیصلے کی توثیق کر دی۔ 4 اپریل کو انھیں راولپنڈی جیل میں پھانسی پر لٹکا دیا گیا۔",
            text_b="ذوالفقارعلی بھٹو پاکستان کے پہلے منتخب وزیراعظم تھے اور آپ نے لاڑکانہ سندھ میں جنم لیا۔ آپ کے والد کا نام سر شاہ نواز بھٹو تھا اور آپ حکومتِ ممبئی میں مشیرِاعلی اور جونا گڑھ کی مسلم ریاستوں میں دیوان کی حثیت سے کام کیا۔ آپ نے برکلے یونیورسٹی ،کیلیفورنیا سے 1950ء پولیٹکس میں گریجویشن کی ڈگری حاصل کی۔ 1952ء میں اصولِ قانون میں ماسٹر آکسفورڈیونیورسٹی سے کیا۔ 1952ء میں ہی بیرسٹری کا پرچہ مڈل ٹمپل لندن سے کامیاب کیا۔ آپ پہلے ایشیائی تھے جو انگلستان کی یونیورسٹی 'ساؤتھمپئین'مں بین الاقوامی قانون کے اْستاد کی حثیت سے فیض ہوئے۔ مسلم لاء کالج کراچی میں بھی کچھ دیر لیکچرر کی حثیت سے فریضہ سر انجام دیے۔ سندھ ہائی کوٹ میں 1953ء میں وقالت شروع کی آپ نے 1967ء میں پاکستان پیپلز پارٹی کے نام سے ایک سیاسی پارٹی بنائی۔ 1970ء کے انتخابات میں آپ کی اس پارٹی نے مغربی پاکستان میں سے زیادہ سیٹس حاصل کیں۔ 1971ء دسمبر میں اس وقت لے جنرل ،یحی خاننے حکومتِ پاکستان آپ کے حوالے کی۔ آپ نے پاکستان کے صدر کی حثیت سے دسمبر1971ء سے 13اگست 1973ء تک اپنے فرئض سر انجام دئیے۔ 1977ء میں ملک میں خانہ جنگی کی حالت قائم ہو گئی جس کی وجہ اس وقت کے انتخابات میں دھاندلیوں تھیں۔ اْس وقت کے چیف آف آرمی سٹاف جنرل محمد ضیاالحق نے ملک میں 5جولائی1977ء کو مارشل لاء لگا دیا۔ ستمبر 1977ء میں آپ کو نواب محمد احمد خان کے قتل کے الزام میں پولیس نے حراست میں لے لیا۔ ہائی کوٹ نے آپ کو 18مارچ 1978ء کو موت کی سزا سْنا دی۔ سپریم کوٹ کی ہائی کوٹ کے فیصلے کی توثیق کے بعد آپ کو 4اپریل1979ء کو راولپنڈی جیل میں پھانسی دے دی گئی۔",
            label=1
        ),
        InputExample(
            guid=12,
            text_a="بینظیر بھٹو پاکستان کی پہلی خاتون وزیراعظم تھیں۔ پہلی بارآپ 1988ء میں پاکستان کی وزیراعظم بنیں لیکن صرف 20 مہینوں کے بعد اس وقت کے صدر پاکستان غلام اسحاق خان نے بے پناہ بدعنوانی کے باعث اپنے خصوصی اختیارت کو استعمال کرتے ہوئے اسمبلی کو ختم کرتے ہوئے نئے الیکشن کروائے۔ بینظیر بھٹو کا تعلق پاکستان پیپلز پارٹی سے تھا۔ ذوالفقار علی بھٹو کی بڑی صاحبزادی سابق وزیراعظم بینظیر بھٹو 21 جون 1953 میں سندھ کے مشہور سیاسی بھٹو خاندان میں پیدا ہوئیں۔ ابتدائی تعلیم کراچی میں حاصل کی Lady Jennings Nursery School اور Convent of Jesus and Mary انھوں نے 15 سال کی عمر بے نظیر بھٹو نے 1973 پولیٹیکل سائنس میں گریجوایشن کر لیا۔ اس کے بعد انھوں نے آکسفورڈ یونیورسٹی میں داخلہ لیا جہاں سے انہوں نے فلسفہ، معاشیات اورسیاسیات میں ایم اے کیا۔ بے نظیر لیاقت باغ میں عوامی جلسے سے خطاب کرنے کے بعد اپنی گاڑی میں بیٹھ کر اسلام آباد آرہی تھیں اس دوران جب وہ پارٹی کارکنوں کے ساتھ اظہار یکجہتی کے لیے گاڑی کی چھت سے باہر نکل رہی تھیں کہ نامعلوم شخص نے ان پر فائرنگ کر دی۔ اس کے بعد بینظیر کی گاڑی سے کچھ فاصلے پر ایک زوردار دھماکہ ہوا اس دھماکے میں بینظیر بھٹو جس گاڑی میں سوار تھیں، اس کو بھی شدید نقصان پہنچا لیکن گاڑی کا ڈرائیور اسی حالت میں گاڑی کو بھگا کر راولپنڈی جنرل ہسپتال لے گیا جہاں وہ زخموں کی تاب نہ لاتے ہوئے جاں بحق ہو گئیں۔",
            text_b="پاکستان کی پہلی خاتون وزیراعظم بینظیربھٹو تھیں۔ 1988ء میں آپ نے پہلی بار وزارتِ اعظمہ کا عہدہ سنبھالا،لیکن صرف 20مہینوں کے قلیل عرصہ میں آپ کی حکومت کو بد عنوانی کے سنگین سبب کی وجہ سے وقت کے صدر غلام اسحاق خان نے ختم کر دیا اور نئےالیکش کیے۔ بینظیر بھٹو کا تعلق سیاسی جماعت پیپلز پارٹی سے تھا،بینظیر بھٹو ذوالفقار علی بھٹوکی بڑی بیٹی تھیں ،آپ نے 21جون 1953ء میں ملک کے ناموراور مشہور سیاسی بھٹو خاندان میں آنکھ کھولی۔ ابتدائی تعلیم اور jennings Nurser 'School کراچی'Convent of jesus and Mary ' Lady کراچی میں حاصل کی۔ 15سال کی عمر میں او لیول اور 1973ء میں پولیٹیکل سائنس میں گریجوایشن مکمل کی۔ اس کے بعد آپ نے فلسفہ ،معاشیات اور سیاسیات میں ایم اے کی ڈگری آکسفورڈ یو نیورسٹی سے حاصل کی۔ بینظیر اسلام آباد کی طرف،لیاقت باغ میں عوامی جلسہ سے خطاب کے بعد اپنی گاڑی میں روان تھیں چلتی گاڑی سے ورکرزکے ساتھ یکجہتی کا اظہار کرنے کے لیے جب آپ گاڑی کی چھت سے باہر کو ہوئیں اِن موقع پر کسی نامعلوم شخص نے آپ کو نشانہ بنا دیا۔ اْسی وقت آپکی گاڑی کے نزدیک ایک زورداردھماکہ ہوا جس سے آپ کے گاڑی کو بھی بہت نقصان پہنچا لیکن آپ کا ڈرائیور گاڑی کو اْسی حالت میں راولپنڈی جنرل ہسپتال لے گیا جہاں آپ خالقِ حقیقی سے جا ملی۔",
            label=1
        ),
        InputExample(
            guid=13,
            text_a="ہمارے ماہرین تاریخ وسیاست نے مادر ملت کو قائداعظم کے گھر کی دیکھ بھال کرنے والی بہن کے حوالے سے بہت بلند مقام دیا ہے لیکن انہوں نے قیام پاکستان اور خصوصاً 1965ء کے بعد کے سیاسی نقشے پر جو حیرت انگیز اثرات چھوڑے ہیں ان کی طرف زیادہ توجہ نہیں دی۔ مادر ملت نے دانتوں کے علاج معالجے میں تخصیص حاصل کی اور کئی سال تک پریکٹس کی اور اس دوران غریبوں کا مفت علاج کیا۔ مادر ملت نے اپنی زندگی کا بیشتر حصہ بے شمار سماجی اور رفاہی اداروں کی سر پرستی میں صرف کیا اور ان کی ترقی اور تعمیر میں دامے 'ورمے 'سخنے مدد کی اس حوالے سے کشمیری مہاجرین کے لئے ان کی خدمات ناقابل فراموش ہیں۔ مادر ملت کی زندگی سے ایک اہم سبق یہ بھی ملتا ہے کہ خواتین کو پرو فیشنل تعلیم سے آ راستہ ہونا چاہئے تاکہ وہ مالی طور پر خود کفیل ہوں۔ مادر ملت کا سب سے اہم کارنامہ پاکستان کو جمہوریت کے راستے پر دوبارہ گامزن کرنا ہے انہوں نے 1965ء کے صدارتی انتخاب میں حصہ لے کر تحریک پاکستان کی ہماگیر عوامی شرکت کی یادیں تازہ کردیں اور مشرقی اور مغربی پاکستان کے مابین یک جہتی اور یگانگت کے جذبوں کو بیدارکردیا۔ وہ قائد اعظم کی وفات کے بعد 19سال تک بقید حیات رہیں اور ملک کے سیاسی 'معاشی 'سماجی اور تعلیمی مسائل کے بارے میں تواتر کے ساتھ اپنی آرا دیتی رہیں ۔",
            text_b="ہمارے سیاست و تاریخ کے ماہرین نے مادرملت محترمہ فاطمہ جناح کو اپنے بھائی قائداعظم کے گھر کی دیکھ بھال کرنے کے حوالے سے بہت بلند مقام دیا ہے۔ لیکن پاکستان کے قیام اور 1965ء کے بعد کے سیاسی نقشے پر جو حیران کْن نشان چھوڑے ہیں ان کی طرف زیادہ رجحان نہیں کیا۔ دانتوں کے علاج معالجے میں مادرِملت نے تخصیص حاصل کی اور پریکٹس کئی سال جاری رکھی۔ اور اپنی پریکٹس کے دوران آپ نے غریبوں کا مفت علاج کیا۔ اپنی زندگی کا بہت زیادہ حصہ مادر ملت نے افلاہی اور سماجی اداروں کی سرپرستی میں گزارا۔ اور ان کی تعمیراور ترقی میں درمے،ورمے،سخنے مدد کی۔ کشمیری مہاجرین کے لیے اس حوالے سے ان کی خدمات نا قابلِ فراموش ہیں۔ ایک اہم مشن جو کہ مادرِملت کی زندگی سے حاصل ہوتا ہے کہ پروفیشنل تعلیم خواتین کو حاصل کرنی چاہیے۔ تاکہ وہ مالی اعتبار سے خود کفیل ہوں۔ جمہوریت کے راستے پر پاکستان کو دوبارہ گامزن کرنا مادرِملت کا سب سے اہم کارنامہ ہے۔ صدارتی انتخابات جو کہ ۱۹۶۵ء میں ہوئے۔ مادرِملت نے اْس میں شمولیت حاصل کرکے تحریکِ پاکستان کی ہماگرِعوامی شرکت کی یادیں زندہ کر دیں۔ اورمشرقی اور مغربی پاکستان کے درمیان یگانگت اور یک جہتی کے جذبوں کو جگا دیا۔ مادرِملت۱۹سال وفاتِ قائداعظم کے بعد زندہ رہیں۔ اور ملک کے تعلیمی،سیاسی،سماجی اور سیاستی مسائل کے بارے میں اپنی رائے مسلسل دیتی رہیں۔",
            label=1
        ),
        InputExample(
            guid=14,
            text_a="تقریباً 25 سال کی عمر میں آپ صلی اللہ علیہ و آلہ وسلم نے شام کا دوسرا بڑا سفر کیا جو حضرت خدیجہ علیہا السلام کے تجارتی قافلہ کے لیے تھا۔ حضرت محمد صلی اللہ علیہ و آلہ وسلم نے ایمانداری کی بنا پر اپنے آپ کو ایک اچھا تاجر ثابت کیا۔ آپ صلی اللہ علیہ و آلہ وسلم دوسرے لوگوں کا مال تجارت بھی تجارت کی غرض سے لے کر جایا کرتے تھے۔ آپ صلی اللہ علیہ و آلہ وسلم یہ خدمات حضرت خدیجہ علیہا السلام کے لیۓ بھی انجام دیا کرتے تھے۔ اس سفر سے واپسی پر حضرت خدیجہ کے غلام میسرہ نے ان کو حضور صلی اللہ علیہ و آلہ وسلم کی ایمانداری اور اخلاق کی کچھ باتیں بتائیں۔ انہوں نے جب یہ باتیں اپنے چچا زاد بھائی ورقہ بن نوفل کو بتائیں تو ورقہ بن نوفل نے کہا کہ جو باتیں آپ نے بتائیں ہیں اگر صحیح ہیں تو یہ شخص یقیناً نبی ہیں. آپ حضرت محمد صلی اللہ علیہ و آلہ وسلم کے اچھے اخلاق اور ایمانداری سے بہت متاثر ہوئیں اور آپ صلی اللہ علیہ و آلہ وسلم کو شادی کا پیغام دیا جس کو آپ صلی اللہ علیہ و آلہ وسلم نے حضرت ابوطالب کے مشورے سے قبول کر لیا۔ اس وقت آپ صلی اللہ علیہ و آلہ وسلم کی عمر 25 سال تھی۔حضرت خدیجہ علیہا السلام قریش کی مالدار ترین اور معزز ترین خواتین میں سے تھیں۔ حضرت خدیجہ علیہا السلام سے آپ صلی اللہ علیہ و آلہ وسلم کی چار بیٹیاں حضرت زینب رضی اللہ تعالٰی عنہ ، حضرت رقیہ رضی اللہ تعالٰی عنہ ، حضرت ام کلثوم رضی اللہ تعالٰی عنہ اور حضرت فاطمہ رضی اللہ تعالٰی عنہ پیدا ہوئیں۔ بعض علمائے اہلِ تشیع کے مطابق حضرت فاطمہ علیہا السلام کے علاوہ باقی بیٹیاں حضرت خدیجہ علیہا السلام کے پہلے خاوند سے تھیں۔",
            text_b="نبی محترمؐ نے حضرت خدیجہؓ کے تجارتی قافلے کے لئے دوسرا بڑا سفرٍ شام کیا تب آپؐ کی عمر 25 سال تھی۔ ایمانداری کی بنیاد پر نبیؐ ایک اچھے تاجر ثابت ہوئے۔ دوسروں کا مال تجارت کی غرض سے آپؐ لے کر جایا کرتے تھے۔ یہ ہی خدمت آپؐ حضرت خدیجہؓ کے لیے بھی کرتے تھے۔ دورانِ سفر حضرت محمدؐ کا اخلاق و ایمانداری دیکھتے ہوئےحضرت خدیجہؓ کے غلام میسرہ نے آکر آپؓ کو بتائی۔ یہ ہی باتیں حضرت خدیجہؓ نے ورقہ بن نوفل کو بتائیں جو کہ اُن کے چچا زاد تھے۔ انہوں نے کہا اگر یہ باتیں درست ہیں تو یقیناً یہ نبی ہیں۔ حضرت خدیجہؓ آپؐ کے اخلاق و ایمانداری سے آپؐ کی گرویدہ ہوگئیں اور آپؐ کو شادی کے لئے پیغام دیا جو کہ اپنے چچا سے مشورہ کر نے کے بعد آپؐ نے منظور کرلیا باوجود اس کے کہ آپؐ کی عمر 25 سال تھی اور حضرت خدیجہؓ قریش کی مالدار اور معزز خواتین میں سے تھیں۔ آپؐ کی اور حضرت خدیجہؓ کی چار بیٹیاں حضرت زینبؓ، حضرت رقیہؓ، حضرت ام کلثومؓ اور حضرت فاطمہؓ تھیں۔ مگر کچھ علمائے اہل ِ تشیع کا کہنا ہے کہ حضرت فاطمہؓ کے علاوہ تمام بیٹیاں حضرت خدیجہ رضی اللہ عنہما کے پہلے شوہر کی تھیں۔",
            label=1
        ),
        InputExample(
            guid=15,
            text_a="آپ 25 دسمبر 1876ء کو وزیر مینشن، کراچی، سندھ میں پید اہوئے، جوکہ اُس وقت بمبئی کا حصہ تھا۔ آپ کا پیدائشی نام محمد علی جناح بھائی رکھا گیا ایک پاکستانی سیاستدان اور آل انڈیا مسلم لیگ کے لیڈر تھے جن کی قیادت میں مسلمانوں نے برطانیہ سے آزادی حاصل کی، یوں پاکستان کا قیام عمل میں آیا اور آپ پاکستان کے پہلے گورنر جنرل بنے۔ سرکاری طور پر پاکستان میں آپ کو قائدِ اعظم یعنی سب سے عظیم رہبر اور بابائے قوم یعنی “قوم کا باپ“ بھی کہا جاتا ہے۔ آغاز میں آپ انڈین نیشنل کانگرس میں شامل ہوئے اور مسلم ہندو اتحاد کے حامی تھے۔ کانگرس سے اختلافات کی وجہ سے آپ نے کانگرس پارٹی چھوڑ دی اور مسلم لیگ کی قیادت میں شامل ہو گئے۔ جناح 1940ء کی قراردادِ پاکستان قرار دادِ لاہور کی روشنی میں مسلمانوں کے لئے ایک علیحدہ ریاست بنام پاکستان بنانے کے لئے مصروفِ عمل ہو گئے۔آخر کار برطانیہ کو پاکستان اور بھارت کی آزادی کے مطالبے کو تسلیم کرنا پڑا۔ 11 ستمبر 1948 پر پاکستان کی آزادی کے صرف ایک سال بعد مر گی",
            text_b="قائد ِاعظم وزیر منشن کراچی میں جو سندھ کا شہراوراُس وقت بمبئی کا حصہ تھا میں 25 دسمبر 1876 کو پیدا ہوئے۔ آپ کا پیدائشی نام محمد علی جناح بھائی ہے۔ آپ ال انڈیا مسلم لیگ کے لیڈر بنے جن کی قیادت میں مسلمانوں نے اپنے آپ کو برطانوی سامراج سے آزاد کروایا اور پاکستان قائم کیا۔ جس کے آپ پہلے گورنر جنرل مقرر ہوئے۔ قائدِ اعظم کا مطلب عظیم رہبر ہے جو کہ حکومت ِ پاکستان نے اُ ن کو دیا بابائے قوم کے لقب سے بھی نوازا۔ جس کے معنی ہیں قوم کا باپ۔ آپ نے سیاست کا آغاز انڈین نیشنل کانگریس سے کیا۔ آپ مسلم و ہندو اتحاد کے حق میں تھے بعد میں آپ نے مسلم لیگ میں شمولیت اختیار کر لی۔ جس کی وجہ آپ کے کانگریس کے ساتھ اختلافات بنے۔ قائدِ اعظم محمد علی جناح 1940ء لاہور میں منعقد ہونے والی قرارداد کی روشنی میں مسلمانوں کے لیے ایک الگ وطن کے لیے کوشاں ہو گئے جس کو پاکستان کا نام دیا گیا۔ بلآخر برطانوی حکومت نے آپ کے مطالبے کے آگے گھٹنے ٹیک دیئے اورپاکستان اور بھارت کو آزاد کر دیا۔ آپ قیام پاکستان کے صرف ایک سال بعد ہی 11 ستمبر 1948ء کودنیا سے رخصت ہو گئے۔",
            label=1
        ),
        InputExample(
            guid=16,
            text_a="بِل گیٹس کا پورا نام ولیم ہنری گیٹس ہے۔ بل گیٹس مائیکروسافٹ کمپنی کا چیئر مین اور دنیا کا امیر ترین شخص ہے.بل گیٹس 1955ء میں امریکہ واشنگٹن کے ایک مضافاتی علاقے نیسٹل میں ایک معمول خاندان میں پیدا ہوا۔ اسے بچپن سے ہی کمپیوٹر چلانے اور اس کی معلومات حاصل کرنے کا شوق تھا۔ آج کی طرح کمپیوٹر نہ اتنے ترقی یافتہ تھے اورنہ ہی کمپیوٹروں کی دنیا، محض ۱۳برس کی عمر میں وہ پروگرامینگ کا ہنرسیکھ چکا تھا۔ اتفاق سے دوستی بھی ایک ایسے لڑکے سے تھی جو خود بھی کمپیوٹروں کا دیوانہ تھا۔ اس دوست کا نام پال ایلن تھا۔ یہ دونوں دوست ہاروڈ یونی ورسٹی میں زیرِ تعلیم تھے مگر کمپیوٹروں کے شوق نے پڑھائی کی بیچ میں سے ہی چھوڑنے پر مجبور کیا اور آخر کار یہ دونوں دوست کالج کو خیر باد کہہ کر ہمہ وقت کمپیوٹروں کی دنیا میں کھو گئے۔ ان کا خاص پروجیکٹ کمپیوٹر کی ایک خاص زبان کو ترتیب دینا تھا جس سے کمپیوٹر کو چلانے میں آسانی ہو۔ ان کی محنت رنگ لائی اور انھوں نے ایک خاص ’’بیسک لنگویج‘‘ مرتب کر لی جس نے کمپیوٹروں کو عام کرنے میں اہم رول ادا کیا۔ یہاں انھوں نے ’’مائکروسافٹ کارپوریشن‘‘ کی نیو ڈالی ۔ اس کے تحت وہ کمپیوٹر کے مختلف سافٹ ویئر تیار کرنا چاہتے تھے جس کی ساری دنیا میں بڑی مانگ تھی کیونکہ پرسنل کمپیوٹر کا استعمال دنیا میں تیزی سے بڑھتا چلا جا رہا تھا۔ ہارڈ ویئر تو مارکیٹ میں بہت دستیاب تھے مگر سافٹ ویئر کی بڑی کمی تھی۔ اس کی زبردست مانگ کے پیش نظر دونوں دوستوں نے اس میدان میں خوب ترقی کی اور آخرکار انھیں ایک ایسا سافٹ ویئر بنانے میں کامیابی ملی جس کو ساری دنیا میں ہاتھوں ہاتھ لیا گیا۔",
            text_b="اگر غریب پیدا ہوئے ہیں تو اس میں آپ کا قصور نہیں اگر غریب ہی مرتے ہیں تو اس میں آپ کا قصور ہے یہ الفاظ دنیا کے امیر ترین شخص کے ہیں۔ جو 14سال تک دنیا کے امیروں میں پہلے نمبر اور اب 2010ء میں دوسرے نمبر پر ہے۔ مگر ہم اسے اس کی سخاوت کی وجہ سے نہیں جانتے بلکہ اُسے مائیکروسافٹ کی وجہ سے جانتے ہیں۔ بل گیٹس 28 اکتوبر 1955ء میں میٹل واشنگٹن میں پیدا ہوئے اِن کا پورا نام ولیم ہنری بل گیٹس سوئم ہے آپ دو بہنوں کے بھائی ہیں اور آپ کے والد ولیم ایچ گیٹس ایک معروف استاد اور قانون دان تھے جبکہ والدہ میری میکسوئل گیٹس ایک ٹیچر تھی جو کہ واشنگٹن یونیورسٹی میں پڑھاتی تھی۔ بل گیٹس شروع سے ہی سوچ بچار کرنے والے بچے تھے جس کی وجہ سے اُن کو ڈاکٹر کو بھی چیک کروایا گیا کہ کہیں وہ بیمار تو نہیں۔بل گیٹس شروع سے ہی کتابی کیڑا تھا۔ اور ہمیشہ اچھے نمبر لیتا تھا۔ اسی لیے اُن کے والدین نے ابتدائی تعلیم کے بعد انہیں ایک چھوٹے پرائیویٹ سکول میں داخل کروایا۔ جس کی واشنگٹن میں عزت و توقیر تھی ۔وہاں وہ کمپیوٹر پر گھنٹوں بیٹھے رہتے اس کے لئے وہاں انہوں نے بوائے اسکاؤ ٹ میں بھی حصہ لے لیا تھا جس میں انہوں نے لائف اسکاؤ ٹ کا خطاب پایا۔ پھر ہائی سکول کے دوران جب اِن کی عمر 16اپریل تھی انہوں نے اپنی زندگی کا پہلا سافٹ ویئر بنایا جسے انہوں نے 4 ہزار میں فروخت کیا۔ اس کے بعد بھی انہوں نے کئی سافٹ ویئر بنائے جس سے انہیں 20 ہزار کی آمدنی ہوئی۔ بل گیٹس نے ہائی سکول میں 1600نمبرز لئے ۔1973ء میں انہوں نے کمپیوٹر سائنس میں بیچلر آف سائنس کی تعلیم شروع کی یہاں اُن کی ملاقات اسٹیو بالمر سے ہوئی۔ جن کے اشتراک سے انہوں نے مستقبل میں اپنا کاروبار شروع کیا۔ دوسرے سال کے آخر میں انہوں نے یونیورسٹی چھوڑ دی یہ کہتے ہوئے کہ آگے پڑھائی جانے والی کتابوں کو وہ پڑھ چکے ہیں۔ جب ہارورڈ یونیورسٹی نے ٹیسٹ لیا تو بل گیٹس 10 نمبر پر آئے اس طرح انہوں نے سافٹ ویئر کی خاطر اپنی تعلیم ادھوری چھوڑ دی۔ اُن کی ترتیب دی ہوئی کمپیوٹر کی بیسک لینگوئج کے مارکیٹ میں آنے کت بعد کمپیوٹر دنیا میں عام ہو گیا۔ جس کے بعد انہوں نے مائیکرو سافٹ کارپوریشن کی بنیاد رکھی۔ جس کی وجہ سے سافٹ ویئر کی مارکیٹ نے ترقی کی اور بل گیٹس پر دولت کی بارش ہونے لگی۔ 2008ء میں بل گیٹس مائیکرو سافٹ کمپنی کو چھوڑ کر دنیا بھر میں بچوں کی شرح اموات کع کم کرنے کے لئے سرگرمِ عمل ہو گئے۔",
            label=0
        ),
        InputExample(
            guid=17,
            text_a="بِل گیٹس کا پورا نام ولیم ہنری گیٹس ہے۔ بل گیٹس مائیکروسافٹ کمپنی کا چیئر مین اور دنیا کا امیر ترین شخص ہے.بل گیٹس 1955ء میں امریکہ واشنگٹن کے ایک مضافاتی علاقے نیسٹل میں ایک معمول خاندان میں پیدا ہوا۔ اسے بچپن سے ہی کمپیوٹر چلانے اور اس کی معلومات حاصل کرنے کا شوق تھا۔ آج کی طرح کمپیوٹر نہ اتنے ترقی یافتہ تھے اورنہ ہی کمپیوٹروں کی دنیا، محض ۱۳برس کی عمر میں وہ پروگرامینگ کا ہنرسیکھ چکا تھا۔ اتفاق سے دوستی بھی ایک ایسے لڑکے سے تھی جو خود بھی کمپیوٹروں کا دیوانہ تھا۔ اس دوست کا نام پال ایلن تھا۔ یہ دونوں دوست ہاروڈ یونی ورسٹی میں زیرِ تعلیم تھے مگر کمپیوٹروں کے شوق نے پڑھائی کی بیچ میں سے ہی چھوڑنے پر مجبور کیا اور آخر کار یہ دونوں دوست کالج کو خیر باد کہہ کر ہمہ وقت کمپیوٹروں کی دنیا میں کھو گئے۔ ان کا خاص پروجیکٹ کمپیوٹر کی ایک خاص زبان کو ترتیب دینا تھا جس سے کمپیوٹر کو چلانے میں آسانی ہو۔ ان کی محنت رنگ لائی اور انھوں نے ایک خاص ’’بیسک لنگویج‘‘ مرتب کر لی جس نے کمپیوٹروں کو عام کرنے میں اہم رول ادا کیا۔ یہاں انھوں نے ’’مائکروسافٹ کارپوریشن‘‘ کی نیو ڈالی ۔ اس کے تحت وہ کمپیوٹر کے مختلف سافٹ ویئر تیار کرنا چاہتے تھے جس کی ساری دنیا میں بڑی مانگ تھی کیونکہ پرسنل کمپیوٹر کا استعمال دنیا میں تیزی سے بڑھتا چلا جا رہا تھا۔ ہارڈ ویئر تو مارکیٹ میں بہت دستیاب تھے مگر سافٹ ویئر کی بڑی کمی تھی۔ اس کی زبردست مانگ کے پیش نظر دونوں دوستوں نے اس میدان میں خوب ترقی کی اور آخرکار انھیں ایک ایسا سافٹ ویئر بنانے میں کامیابی ملی جس کو ساری دنیا میں ہاتھوں ہاتھ لیا گیا۔",
            text_b="مائیکرو سافٹ کمپنی کا چیئرمین اور دنیا کا امیر ترین شخص بِل گیٹس ہے ۔ بِل گیٹس کاپورا نام ولیم ہنری گیٹس سوئم ہے ۔ جو کہ 28 اکتوبر 1955ء میںنیسٹل واشنگٹن میں پیدا ہوئے ۔ آپ کے والد کا نام سر ولیم ایچ گیٹس تھا جو معروف استاد اور قانون دان تھے اور والدہ میری میکسوٹل گیٹس ٹیچر تھی جو واشنگٹن یونیورسٹی میں پڑھایا کرتی تھیں ۔بل گیٹس بچپن سے ہی انوکھی طبیعت کا مالک تھا۔ اِس کے مشاغل دوسرے بچوں سے مختلف تھے لہذا ابتدائی تعلیم کے بعد اس کے والدین نے اسے نسائڈ کے ایک چھوٹے پرائیویٹ سکول میں داخل کروا دیا جس کا نام واشنگٹن میں عزت اور توقیر سے لیا جاتا تھا ۔ اچھی تربیت اور تعلیم کی بناء پر بل گیٹس کو شروع ہی سے لوگوں کی خدمت کرنا اچھا لگتا تھا بلکہ ایک طرح کا شوق تھا ۔ اپنے اسی شوق کی خاطر وہ بوائے اسکاؤٹ میں بھرتی ہو گیا اس کی دلچسپیوں کی وجہ سے اسکول نے اسے لائف اسکاؤٹ کے خطاب سے نوازا ۔ کم عمری میں اس اعزاز کو حاصل کرنا معمولی بات نہیں تھی ۔ہائی سکول کی تعلیم کے دوران 16 سال کی عمر میں ایک سافٹ ویئر بنایا جسے اس نے 4 ہزار ڈالر میں فروخت کیا۔ 18 برس کی عمر میں اس نے ایک پورٹ لینڈ اور یگان کی ایک کمپنی کوبول کو ایک پے رول سسٹم بنا کر دیا ۔ جو کمپنی کے لئے کافی مفید ثابت ہوا ۔ہارورڈ یونیورسٹی میں تعلیم حاصل کرنے کے دوران بل گیٹس اور اِ س کا دوست پال ایلن کمپیوٹروں میں بے انتہا دلچسپی کی وجہ سے تعلیم چھوڑ کر کمپیوٹروں میں لگ گئے ۔ دونوں دوستوں نے مل کر محنت کی اور بلآخر ایک سافٹ ویئر کو ترتیب دیا جو بہت بسند کیا گیا اور اِن کی محنت کو بہت سراہا گیا مائیکرو سافٹ کمپنی کے نام تلے انہوں نے اور بھی بہت ایسی کدمات سر انجام دیں",
            label=0
        ),
        InputExample(
            guid=18,
            text_a="ملکہ وکٹوریہ سلطنت برطانیہ کی ملکہ تھی۔ ملکہ وکٹوریہ بروز سوموار 24 مئی 1819 ء کو صبح 04:15 پر کینسنگٹن محل لندن میں پیدا ہوئی۔ منگل20 جون1837ء کو 18سال 28 دن کی عمر میں ملکہ برطانیہ بنی اور تا وفات ملکہ رہی۔ 10فروری 1840ء کو شہزادہ البرٹ سے رشتہ ازدواج میں منسلک ہو گئی۔ 14دسمبر 1861ء کو پرنس البرٹ تپ محرقہ کا شکار ہو کر وفات پاگیا۔ اس کے بعد ملکہ نے چالیس سال حالت بیوگی میں اداس تنہا گزارے۔ ملکہ کا کل دور اقتدار 63 سال 7 مہینے 3 دن ہے، اس لحاظ سے وہ سب سے زیادہ دیر تک اقتدار میں رہنے والی برطانوی حکمران تھی۔ بروز منگل 22 جنوری 1901ء کو شام06:30 پر وفات پائی اُس وقت کل عمر 81 سال 8 مہینے تھی۔ سوموار 4 فروری 1901ء کو فراگمور وِنڈسر میں تدفین کی گئی۔ ملکہ وکٹوریہ کے دور میں سلطنت برطانیہ اپنی طاقت اور عالمی اثر و رسوخ میں اپنی انتہا پر تھی اور یہ دور بہت تبدیلیوں کا دور تھا۔ ملکہ کے 9 بچے تھے۔ انگریز دنیا ميں جہاں بھی گئے اپنی اس ملکہ کو نہ بھولے آسٹریلیا کے ایک صوبے کا نام وکٹوریا ہے۔",
            text_b="ملکہ وکٹوریہ جنھوں نے برطانیہ میں سب سے ذیادہ برطانوی حکمرانی کی 1819ء کو برطانہ میں پیدا ہو ئیں۔ ملکہ وکٹو ریہ کو پینٹیگ کا بہت شوق تھا،آپ کی پینٹیگ آج بھی دنیا میں بہت مشہور اور اپنی ایک خاص اہمیت رکھتی ہے۔ ایک آئل پینٹنگ جس میں ملکہ وکٹوریہ کے ساتھ انکے ایک ملازم کو بھی دیکھا گیا ہے۔ آپ نے یہ پینٹنگ جان براؤن کو ان کی سالگرہ پر دی تھی۔وہ بہت اچھے اور سمجھدار ہیں یہ وہ الفاظ ہیں جو آپ نے اپنے شوہر کے لیے نہیں بلکہ اپنے ملازم جان براؤن کے لئے کہے آپ ان کی بہت عزت کرتی تھیں۔وہ بھی 1833 ء کو وفات پا گئے۔ ملکہ وکٹوریہ کو کریم نامی نوجوان نے سیکھائی جو کہ ڈاکڑ ٹیلر کے ساتھ ملازمت کے طور پر آئے تھے۔کریم اپنی ڈائری میں لکھتے ہیں کہ وہ لمحات بیان کرنے کی کوشش کر رہا جو 1887ء کی گولڈن جو بلی سے لے کر 1897ء کی ڈائمنڈ جوبلی تک انھوں نے ملکہ کے محل میں گذارے ۔ ملکہ وکٹو ریہ نے 63سال حکو مت کی ان کی حکو مت میں برطانوی سلطنت اپنے عروج پر تھیں۔آپ کی شادی جرمنی کذن شہزادہ ایلیرٹ سے ہوئی۔ ملکہ وکٹوریہ کو اپنے شو ہر سے بہت محبت تھی جبکہ ان کے شو ہر شدید ٹائیفیڈ کی وجہ سے 1816ء کو وفات پا گئے ۔اپنے شوہر کی وفا ت کے بعد آپ بہت اداس رہتی تھیں۔ اور 40 سال تک آپ تنہا رہیں۔ آپ کو 6 مرتبہ قتل کرنے کی کو شش کی گئی۔ آخر کار وزیرِاعظم نے ملکہ وکٹوریہ کو دوبارہ برطانوی سلطنت کو چلانے پر راضی کیا۔ اور آہستہ آہستہ آپ نے بر طانیہ سلطنت کو پھر سے اپنے عمل میں کیا۔ آپ کے ایک ملازم عبدالکر یم اپ کے شوہر کی وفات کے بعد بھی آپ کے ساتھ رہے اور ہر چیز میں آپ کو مشورہ دیتے۔ انھوں نے مسلمانوں کو ہندوستان سے الگ کرنے کا مشورہ دیا وہ ہمیشہ مسلمانوں کی رہنمائی میں بولتا کیونکہ وہ مسلمان تھا۔ 1867 ء کو آپ کو اِیمپریس آف انڈیا کا خطاب دیا گیا۔ ملکہ وکٹوریہ کو برطانوی حکو مت کرتے ہو ئے 60 اور 50 سال ہو گئے تو اِن کی گولڈن اور ڈائمنڈ جوبلی بڑی شان سے منائی گئی۔ ملکہ وکٹوریہ کے بیٹے کنگ ایڈورڈ نے جو کہ الکریم کو ناپسند کرتا تھا ان کو ہر طرف کر دیا اور ہندو ستان واپس بھیج دیا۔ ملکہ وکٹوریہ 1837ء سے لے کر آخری سال 1901ء تک برطانیہ کی حکمران رہیں۔آخر کار آپ نے 82 سال کی عمر میں وفات پائی۔",
            label=0
        ),
        InputExample(
            guid=19,
            text_a="آپ 25 دسمبر 1876ء کو وزیر مینشن، کراچی، سندھ میں پید اہوئے، جوکہ اُس وقت بمبئی کا حصہ تھا۔ آپ کا پیدائشی نام محمد علی جناح بھائی رکھا گیا ایک پاکستانی سیاستدان اور آل انڈیا مسلم لیگ کے لیڈر تھے جن کی قیادت میں مسلمانوں نے برطانیہ سے آزادی حاصل کی، یوں پاکستان کا قیام عمل میں آیا اور آپ پاکستان کے پہلے گورنر جنرل بنے۔ سرکاری طور پر پاکستان میں آپ کو قائدِ اعظم یعنی سب سے عظیم رہبر اور بابائے قوم یعنی “قوم کا باپ“ بھی کہا جاتا ہے۔ آغاز میں آپ انڈین نیشنل کانگرس میں شامل ہوئے اور مسلم ہندو اتحاد کے حامی تھے۔ کانگرس سے اختلافات کی وجہ سے آپ نے کانگرس پارٹی چھوڑ دی اور مسلم لیگ کی قیادت میں شامل ہو گئے۔ جناح 1940ء کی قراردادِ پاکستان قرار دادِ لاہور کی روشنی میں مسلمانوں کے لئے ایک علیحدہ ریاست بنام پاکستان بنانے کے لئے مصروفِ عمل ہو گئے۔آخر کار برطانیہ کو پاکستان اور بھارت کی آزادی کے مطالبے کو تسلیم کرنا پڑا۔ 11 ستمبر 1948 پر پاکستان کی آزادی کے صرف ایک سال بعد مر گی",
            text_b="پاکستان کی ایک اعظیم حستی جن کی بدولت پاکستان قیامِ وجود میں آیا۔جن کو بانئی پاکستان کہا جاتا ہے۔وہ قائداعظم محمد علی جناح 25دسمبر1876ء کو کراچی میں پیدا ہوئے۔آپ کے والد کا نام پونجا جناح تھا۔آپ کے والد ایک مالدار تاجر تھے۔آپ نے 16سال کی عمر میں میٹرک کا امتحان پاس کیا۔1895 ء کو لندن سے آپ نے قانون کی ڈگری حاصل کی ۔برطانیہ واپس آکر وکالت کا آغاز کیا۔آپ نے 1896ء میں انڈین نیشنل کانگرس میں شمولیت اختیار کی۔اس وقت وہ ہندوں کی سب سے بڑی سیاسی جماعت تھی۔وقت گزارنے کے ساتھ ساتھ آپ کو معلوم ہو گیا کہ ہندو مسلمانوں پر جان بوجھ کے ظلم کرتے اور ان کو ان کے حقوق سے محروم کرنے میں لگے رہتے۔آپ نے ہندوں کو بہت سمجھایا اور آخر کار اختلافات کی وجہ سے ہندو کانگرس سے علیحدگی اختیار کر کے مسلم لیگ میں شامل ہو گئے۔اور مسلمانوں کو ان کے حقوق دلانے کے لئے جدوجہد کرنے میں مصروف ہوگئے۔آپ نے مسلمانوں کو یقین دلایا کہ وہ ہندوں سے الگ قوم ہیں۔اور ان کے لئے ایک الگ ملک ہونا چاہیے جہاں وہ اپنے مطابق زندگی گزار سکیں۔علامہ اقبال آپ کے بہت اچھے دوست تھے۔آپ نے علامہ اقبال کے ساتھ مل کر مسلمانوں کو جگانے میں بہت کوشش کی۔ان میں اپنے تحفظ وسکون اور الگ ملک کے لئے لڑنے کے لیے جذبہ پیدا کیا۔1916ء میں آل انڈیا مسلم لیگ کے اجلاس جو کہ لکھنو میں آپکی بدولت 30 اور 31 دسمبر کو منقعد ہوئے۔جس میں مسلمانوں کے لئے ایک جداگانہ انتخابات کو پیش کیا۔1930 ء کو آل انڈیامسلم لیگ کے اجلاس میں آ پ نے ایک میزگول کانفرنس تشکیل دی۔ جس میں آپ نے برطانیہ سے پاکستان اور بھارت کو الگ ہونے کا مطالبہ پیش کیا۔ 23 مارچ 1940 ء کو مسلم لیگ ایک الگ ریاست کے طور پرنمودار ہوئی جس کانام پاکستان رکھا گیا۔مسلمانوں کے تحفظ ان کے رہن سہن اور لاس سب ہندوں سے مختلف تھا۔اب وہ یہاں اپنے اور اسلام کے مطابق زندگی گزار سکتے ہیں۔1948 ء میں دن رات کی سخت محنت کی وجہ سے آپ بہت بیمار ہوگئے۔11 ستمبر1948ء کو آپ نے وفات پائی۔",
            label=0
        ),
        InputExample(
            guid=20,
            text_a="اعزازِ نشان حیدر حاصل کرنے والے پائلٹ آفیسر کراچی میں پیدا ہوئے۔ منہاس راجپوت گوت گھرانے کے چشم چراغ تھے۔ 1968ء میں سینٹ پیٹرک سکول کراچی سے سینئر کیمبرج کیا۔ خاندان کے متعدد افراد پاکستان کی بری بحری اور فضائی افواج میں اعلیٰ عہدوں پر فائز تھے۔ فروری 1971ء میں پشاور یونیورسٹی سے انگریزی، ائیر فورس لا ملٹری ہسٹری الیکڑونکس موسمیات جہازرانی ہوائی حرکیات وغیرہ میں بی۔ ایس۔ سی کیا۔ اور اگست 1971ء میں پائلٹ آفیسر بنے۔20 اگست 1971ء کو راشد کی تیسری تنہا پرواز تھی۔ وہ ٹرینر جیٹ طیارے میں سوار ہوئے ہی تھے کہ ان کا انسٹرکٹر سیفٹی فلائٹ آفیسر مطیع الرحمان خطرے کا سگنل دے کر کاک پٹ میں داخل ہوگیا اور طیارے کا رخ بھارت کی سرحد کی طرف موڑ دیا۔ اگلے پانچ منٹ راشد اور انسٹرکٹر کے درمیان کشمکش میں گزرے اور اسی کش مکش کے دوران طیارہ زمین پر گر کر تباہ ہوگیا۔ راشد نے شہادت کا درجہ پایا اور انھیں اس عظیم کارنامے کے صلے میں سب سے بڑا فوجی اعزاز نشانِ حیدر دیا گیا۔",
            text_b="جرات اور بہادری کی عظیم داستان پاکستان کا اعلی ترین فوجی اعزاز نشان حیدر حاصل کرنے والے پائلٹ راشد منہاس 17 فروری 1951ء کو کراچی میں پیدا ہوئے۔ راشد منہاس کا تعلق راجپوت گھرانے سے تھا۔ 1968ء میں سینٹ پیٹرک سکول کراچی سے سینئسر کیمبرج کیا۔ پاکستان ائرفورس میں شمولیت اختیار کرلی۔ 1971ء میں وہ پائلٹ آفسر بن گئے خاندان کے متعدد افرا د پاکستان کی بری بحری اور فضائی افواج میں اعلی عہدوں پر فائز تھے۔ زاشد منہاس نے ابتدائی تعلیم جیس اسکول اور کوئین میری اسکول سے حاصل کی انہوں نے اپنا آئیڈئل فوجی زندگی کو بنایا۔ اور اپنے ماموں ونگ کمانڈر سعید سے جزباتی وابستگی کی بنا پر فضائیہ کا انتخاب کیا ۔تربیت کے لئے پہلے کواہٹ اور پھر پاکستان ائیر فورس اکیڈمی رسالپور بھیجے گئے۔ جون 1970ء میں انہوں نے پشاور یونیورسٹی سے بی ایس سی میں فرسٹ ڈ ویثرن میں پاس کیا۔ فروری 1971ء میں پشاور یونیورسٹی سے انگریزی،ائرفورس،ملٹری بسٹری،الیکٹرونکس ،موسمیات ،جہاز رانی بوائی حرکیات وغیرہ میں بی ایس سی کی۔ بعد ازاں تربیت کے لئے کراچی بھیج دیا گیا اور اگست 1971ء کو راشد منہاس کی تیسری تنہا پرواز تھی۔ وہ ٹرینر جیٹ طیارے میں سوار ہوئے تھے کی انکا انسٹریکٹر سیفٹی فلائٹ آفسر مطیع الرحمن خطرے کا سگنل دے کر کاک پٹ میں داخل ہو گیا۔ اور طیارے کارخ بھارت کی سرحد کی طرف موڈ دیا۔ پانچ منٹ راشد اور انسٹرکٹر کے درمیان کشمکش میں گزرے اور اسی کشمکش کے دوران طیارہ زمین پر گر گیا۔ اس طیارے کی تباہی کی ایک بڑی وجہ یہ تھی کے بھارت اس طیارے کو اغواہ کرنا چاہتا تھا۔ فضائیہ کے خفینہ رازوں کو بھی بھارت کی سرحد میں داخل ہو نے سے بچایا۔ اس کارنام کی بدولت آپ کو بڑا فوجی اعزاز 'نشانِ حیدر 'دیا گیا۔ راشد منہاس نے بہادری اور جرات کے عظیم ترین کارنامے انجام دئیے۔",
            label=0
        ),
        InputExample(
            guid=21,
            text_a="ہمارے ماہرین تاریخ وسیاست نے مادر ملت کو قائداعظم کے گھر کی دیکھ بھال کرنے والی بہن کے حوالے سے بہت بلند مقام دیا ہے لیکن انہوں نے قیام پاکستان اور خصوصاً 1965ء کے بعد کے سیاسی نقشے پر جو حیرت انگیز اثرات چھوڑے ہیں ان کی طرف زیادہ توجہ نہیں دی۔ مادر ملت نے دانتوں کے علاج معالجے میں تخصیص حاصل کی اور کئی سال تک پریکٹس کی اور اس دوران غریبوں کا مفت علاج کیا۔ مادر ملت نے اپنی زندگی کا بیشتر حصہ بے شمار سماجی اور رفاہی اداروں کی سر پرستی میں صرف کیا اور ان کی ترقی اور تعمیر میں دامے 'ورمے 'سخنے مدد کی اس حوالے سے کشمیری مہاجرین کے لئے ان کی خدمات ناقابل فراموش ہیں۔ مادر ملت کی زندگی سے ایک اہم سبق یہ بھی ملتا ہے کہ خواتین کو پرو فیشنل تعلیم سے آ راستہ ہونا چاہئے تاکہ وہ مالی طور پر خود کفیل ہوں۔ مادر ملت کا سب سے اہم کارنامہ پاکستان کو جمہوریت کے راستے پر دوبارہ گامزن کرنا ہے انہوں نے 1965ء کے صدارتی انتخاب میں حصہ لے کر تحریک پاکستان کی ہماگیر عوامی شرکت کی یادیں تازہ کردیں اور مشرقی اور مغربی پاکستان کے مابین یک جہتی اور یگانگت کے جذبوں کو بیدارکردیا۔ وہ قائد اعظم کی وفات کے بعد 19سال تک بقید حیات رہیں اور ملک کے سیاسی 'معاشی 'سماجی اور تعلیمی مسائل کے بارے میں تواتر کے ساتھ اپنی آرا دیتی رہیں ۔",
            text_b="قائدِ اعظم محمد علی جناح کو کون نہیں جانتا پاکستانیوں کے لئے وہ عظیم انسان ہیں جنہوں نے انہیں ایک پہچان دی ہے۔ اوراگر قائدِ اعظم کی زندگی پرغور کیا جائے تو یہ سب کچھ جو انہوں نے کیا وہ فاطمہ جناح کے بغیر نہ ہو سکتا تھا۔ اُن کی زندگیوں پر غور کیا جائے تو یوں محسوس ہوتا کہ محمد علی جناح کی زندگی فاطمہ جناح کی زندگی تھی اور فاطمہ جناح کی زندگی محمد علی جناح کی زندگی تھی۔ عمروں کے فرق اور فاصلے کے باوجود فاطمہ جناح کی زندگی قائدِ اعظم کی زندگی کے ساتھ اس قدر متوازی چلتی ہے کہ قائدِ اعظم کی زندگی ایک نہیں شخصیت کی نہیں دو شخصیات کی سوانح ہے۔ فاطمہ جناح اپنے عظیم بھائی کی تصویر تھیں۔ فاطمہ جناح کی اپنے بھائی کی طرح قربانیوں اور محنت و جدوجہد کی وجہ سے ہی انہیں مادر ملت کہا جاتا ہے۔ محترمہ فاطمہ جناح جولائی 1891ء میں اپنے آبائی گھر نیونہم روڈ کراچی میں پیدا ہوئی۔ آپ قائد ِاعظم سے عمر میں 16 برس چھوٹی تھی جب قائدِ اعظم وکالت کی تعلیم کے لئے 1892ء میں بیرون ملک تشریف لے جا رہے تھے تو آپ ایک سال کی تھی۔ دو سال کی عمر میں آپ کی والدہ مٹھی بائی کا انتقال ہو گیا۔1895ء میں آپ کے والد مستقل بمبئی تشریف کے گئےاور دس سال کی عمر میں آپ والد کی شفقت سے بھی محروم ہوگئی۔ تب قائدِ اعظم نے آپ کی ست پرستی کی اور انہیں ابتدائی تعلیم کے لئے سینٹ میٹرک ہائی سکول 1902ء میں کفنڈالہ میں داخل کرودیا۔ 1910ء میں آپ نے میڑک اور 1913 ء میں سینئر کیمبرج کا امتحان پرائیویٹ طالبہ کی حیثیت سے پاس کرنے کے بعد 1922ء میں آپ ڈینٹیسٹ بن گئی اور 1923ء میں عبدالرحمٰن اسٹریٹ میں بمبئی میں پریکٹس شروع کردی کچھ سال آپ نے انسانیت کی خدمت کی اور 1929ء میں قائدِ اعظم کی بیوی رتن بائی کی وفات کے بعد انہوں نے صرف قائدِ اعظم کی خدمت کرنے کا فیصلہ کر لیا۔ اُنہوں نے ہر میدان میں بھائی کاساتھ دیا۔ چاہے وہ قیام پاکستان کی جدوجہد میں ہو یا قیام پاکستان کے بعد کی جدوجہد ہو فاطمہ جناح اپنے بھائی کے قدم سے قدم ملا کر سبز ہلالی پرچم ہاتھ میں ہےگلیوں کوچوں اور بازاروں میں قیام پاکستان کے لئے جدو جہد کرتی رہی اور تکلیف برداشت کرتی رہی۔ 7 اگست کو 56 سال کے بعد دہلی چھوڑ کر کراچی منتقل ہو گئیں۔ یہاں انہوں نے پاکستانیوں کی سیاسی ، اقتصادی ترقی کے لئے محنت کرتی رہی آپ نے عورت کی تعلیم پر خاص توجہ دی اور اِن کو مردوں کے شانہ بشانہ محنت کرنے کی ترغیب دی آپ کی کشمیری مجاہدین کے لئے کی گئی خدمات بھی ناقابل فراموش ہیں ۔آپ عورتوں کے لئے مشعل ِ راہ ہیں۔ آپ کی اپنے بھائی سے محبت کا یہ عالم ہے کہ وہ اُن کی وفات کے بعد ہی 8 اور 9 جولائی کی درمیانی شب 1968ء میں قصر فاطمہ کلفٹن میں 76 سال کی عمر میں اپنے خالق حقیقی سے جا ملی۔",
            label=0
        ),
        InputExample(
            guid=22,
            text_a="حضرت زینب سلام اللہ علیہا امام علی علیہ السلام اور حضرت فاطمہ سلام اللہ علیہا کی بیٹی یعنی حضرت محمد صلی اللہ علیہ و آلہ و سلم کی نواسی تھیں۔ وہ 5 جمادی الاول 6ھ کو مدینہ میں پیدا ہوئیں۔ واقعہ کربلا کے بعد حضرت زینب سلام اللہ علیہا کا کردار بہت اہم ہے۔ واقعہ کربلا کے بعد وہ دمشق لے جائی گئیں جہاں یزید کے دربار میں دیا گیا ان کا خطبہ بہت مشہور ہے۔ آپ نے اپنے خطبے میں فرمایا۔ یزيد اگر چہ حادثات زمانہ نے ہمیں اس موڑ پر لا کھڑا کیا ہے اور مجھے قیدی بنایا گیا ہے لیکن جان لے میرے نزدیک تیری طاقت کچھ بھی نہیں ہے۔ خدا کی قسم، خدا کے سوا کسی سے نہیں ڈرتی اس کے سوا کسی اور سے گلہ و شکوہ بھی نہیں کروں گی۔ ہم اہل بیت پیغمبر سے دشمنی کے لئے تو جتنی بھی سازشیں کرسکتا ہے کر لے لیکن خدا کی قسم تو ہمارے نام کو لوگوں کے دل و ذہن اور تاریخ سے نہیں مٹا سکتا اور اسی طرح تو اپنے دامن پر لگے ننگ و عارکے بدنما داغ کو بھی نہیں دھو سکتا، خدا کی نفرین و لعنت ہوظالموں اور ستمگروں پر۔ حضرت زینب اپنے بھائی امام حسین کی شہادت کے تقریبا' ڈیڑھ سال بعد 15 رجب المرجب 62 ہجری کودرجہ شہادت پر فائز ہوئیں۔ ان کا روضہ اقدس شام کے دارالحکومت دمشق میں ہے۔",
            text_b="حضرت محمد ؐکی نواسی یعنی حضرت فاطمہؑ اور امام علی علیہ السلام کی بیٹی حضرت زینبؑ ہیں۔ آپؑ مدینہ میں 5 جمادی الاول 6ھ کو پیدا ہوئیں۔ حضرت زینبؑ کا کردار کربلا کے واقعے کے بعد بہت اہم ہے۔ جس کے بعد آپؑ کو دمشق لے جایا گیااور یزید کے دربار میں پیش کیا گیا جہاں آپؑ نے خطبہ دیا جو بہت مشہور ہے۔خطبہ میں آپؑ نے فرمایا۔ زمانے کے حادثوں نے مجھے اِس موڑ پر لاکھڑا کیا ہے کہ میں قیدی بنا دی گئی ہوں۔ لیکن یزید میرے قریب تیری طاقت کچھ بھی نہیں ہے۔ خُدا کی قسم اُس کے سوا نا کسی سے ڈرتی ہوں اور نہ ہی اُس کے علاوہ کسی سے گلہ شکوہ بھی کروں گی۔یزید تو ہم اہلِ بیت پیغمبرؐ سے جتنی بھی دشمنی میں سازشیں کر سکتا ہے کر لے۔ لیکن خداکی قسم تو تاریخ اور لوگوں کے دل و زہن سے ہمیں ختم نہیں کر سکتا۔تو اپنے سینے پر لگے ننگ و عار کے بد نمانشانوں کو بھی نہیں صاف کر سکتا۔ خدا کی لعنت و ملامت ہو ستمگروں اور ظلم کرنے والوں پر۔حضرت زینبؑ کو شہادت کا مرتبہ اپنے بھائی امام حسین سے شہادت کے تقریباڈیڑھ سال بعد 15 رجب المرجب 62 ہجری کو ملا۔ شام کے دارالحکومت دمشق میں آپ کا روضہ اقدس ہے۔",
            label=0
        ),
        InputExample(
            guid=23,
            text_a="عظیم پاکستانی قوال، موسیقار اور گلوکار فیصل آباد میں پیدا ہوئے۔ آپ کے والد فتح علی خان اور تایا مبارک علی خان اپنے وقت کے مشہور قوال تھے۔ ان کے خاندان نے قیام پاکستان کے وقت مشرقی پنجاب کی ضلع جالندھر سے ہجرت کر کے فیصل آباد میں سکونت اختیار کی تھی۔ استاد نصرت فتح علی خان نے اپنی تمام عمر قوالی کے فن کو سیکھنے اور اسے مشرق و مغرب میں مقبول عام بنانے میں صرف کر دی۔ انہوں نے صوفیائے کرام کے پیغام کو دنیا کے کونے کونے تک پہنچایا اور ان کے فیض سے خود انہیں بے پناہ شہرت نصیب ہوئی۔پاکستان میں نصرت فتح علی خان کا پہلا تعارف اپنے خاندان کی روایتی رنگ میں گائی ہوئی ان کی ابتدائی قوالیوں سے ہوا۔ ان کی مشہور قوالی ’علی مولا علی‘ انہں دنوں کی یادگار ہے۔ بعد میں انہوں نے لوک شاعری اور اپنے ہم عصر شعراء کا کلام اپنے مخصوص انداز میں گا کر ملک کے اندر کامیابی کے جھنڈے گاڑے عام طور پر قوالی سے لگاؤ نہ رکھنے والے طبقے کو اپنی طرف راغب کیا اور یوں نصرت فتح علی خان کا حلقہ اثر وسیع تر ہو گیا۔",
            text_b="فن قوالی کو نئی جہتوں سے متعارف کرانے والے استاد نصرت فتح علی خان کی یادیں آج بھی ان کے چاہنے والوں کے دلوں میں زندہ ہیں۔ 13 اکتوبر 1948ء کو فیصل آباد میں پیدا ہوئےآپ کا اصل نام پرویز ہے لیکن موسیقی کی دنیا میں نصرت فتح علی خان کے نام سے متعارف ہوئے۔پاکستان میں نصرت فتح علی خان کا تعارف اپنے خاندان کی روائتی رنگ میں گائی ہوئی ان کی ابتدائی قوالیوں سے ہوا۔ گائیکی کی تعلیم اپنے والد سے حاصل کی تاہم، اُن کے فوت پو جانے کے بعد اپنے چچا کی شاگردی اختیار کر لی۔ اور وقت گزرنے کے ساتھ ساتھ انہوں نے صوفیائے کرام کے پیغام کو دنیا کے کونے کونے تک پہنچایا جن کے فیض سے وہ خود بھی شہرت کے آسمان پر چمکتے ایسے ستارے بن گئے جن سے ہزاروں لوگوں نے رہنمائی حاصل کی۔ آپ نے پاکستان میں ہی نہیں بلکہ ہندوستان ، کینیڈا اور دوسرے ملکوں میں بھی اپنی صلاحیتوں کا لوہا منوایا۔ آپ کےایک وقت میں 25 البم ریلز ہوئے جن کی شہرت نے انہیں ' گینز بک آف ورلڈ ریکارڈ' میں جگہ دلوائی۔قوالی کی طرف توجہ نہ دینے والے طبقے کو بھی آپ نے اپنی موسیقی کے فن سے اپنی طرف راغب کر لیا۔ 48 سال کی عمر میں اپنے کیرئیر کے عروج کے دوران گردوں کے عارضے کی وجہ سے اپنے 16 اگست 1997 ء میں وفات پائی۔ آپ کی موت لوگوں کے لئے ایک سانحہ سے کم نہ تھی۔",
            label=0
        ),
        InputExample(
            guid=24,
            text_a="ملکہ وکٹوریہ سلطنت برطانیہ کی ملکہ تھی۔ ملکہ وکٹوریہ بروز سوموار 24 مئی 1819 ء کو صبح 04:15 پر کینسنگٹن محل لندن میں پیدا ہوئی۔ منگل20 جون1837ء کو 18سال 28 دن کی عمر میں ملکہ برطانیہ بنی اور تا وفات ملکہ رہی۔ 10فروری 1840ء کو شہزادہ البرٹ سے رشتہ ازدواج میں منسلک ہو گئی۔ 14دسمبر 1861ء کو پرنس البرٹ تپ محرقہ کا شکار ہو کر وفات پاگیا۔ اس کے بعد ملکہ نے چالیس سال حالت بیوگی میں اداس تنہا گزارے۔ ملکہ کا کل دور اقتدار 63 سال 7 مہینے 3 دن ہے، اس لحاظ سے وہ سب سے زیادہ دیر تک اقتدار میں رہنے والی برطانوی حکمران تھی۔ بروز منگل 22 جنوری 1901ء کو شام06:30 پر وفات پائی اُس وقت کل عمر 81 سال 8 مہینے تھی۔ سوموار 4 فروری 1901ء کو فراگمور وِنڈسر میں تدفین کی گئی۔ ملکہ وکٹوریہ کے دور میں سلطنت برطانیہ اپنی طاقت اور عالمی اثر و رسوخ میں اپنی انتہا پر تھی اور یہ دور بہت تبدیلیوں کا دور تھا۔ ملکہ کے 9 بچے تھے۔ انگریز دنیا ميں جہاں بھی گئے اپنی اس ملکہ کو نہ بھولے آسٹریلیا کے ایک صوبے کا نام وکٹوریا ہے۔",
            text_b="اٹھارہ سال کی عمر میں ملکہ بننے والی ملکہ وکٹوریہ 1819ء کو برطانیہ میں پیدا ہوئیں۔ اِ س کے دورِ حکومت میں برطانیہ کی سلطنت نے اور وسعت اختیار کی۔ اپنی عوام کی ہر وقت ضرورت کا خیال رکھتے ہوئے وکٹوریہ نے اپنی عوام کے لوگوں میں گہری دلچسپی لی جس کے نتیجے میں اُس کو عوام کی طرف سے بہت عزت اور احترام حاصل ہوا۔ جرمن کزن پرنس البرٹ سے شادی ہوئی جو اچانک ٹائیفائیڈ کی وجہ سے 1861ء میں فوت ہوگئے۔ اپنے خاوند کی موت کے سوگ میں تاحیات تک انہوں نے سیاہ لباس زیب تن کیا۔ اپنے دور حکومت میں اُن کو چھے مرتبہ قتل کرنے کی کوشش کی گئی۔ خاوند کی وفات کے بعد سکاٹش نوکر جان براؤن اور 24 سالہ ہندوستانی نوکر عبدالکریم کے ساتھ تعلقات میں رہیں۔ ایمپریس آف انڈیا خطاب آپ کو 1867ء میں ملا۔ 50 اور 60 سال تخت نشین رہنے کے بعد آپ کی گولڈن اور ڈائمنڈ جوبلی منائی گئی۔ملکہ وکٹوریہ 1837ء سے اپنی زندگی کے آخری سال 1901ء تک برطانیہ کی ملکہ رہیں جو کہ طویل ترین زمانہِ حکومت ہے۔ آپ نے 82 سال کی عمر میں 1901ء میں وفات پائی۔",
            label=0
        ),
        InputExample(
            guid=25,
            text_a="پاکستان کے پہلے منتخب وزیر اعظم لاڑکانہ سندھ میں پیدا ہوئے۔ انکے والد سر شاہ نواز بھٹو مشیر اعلی حکومت بمبئی اور جونا گڑھ کی مسلم ریاست میں دیوان تھے۔ ذوالفقار علی بھٹو نے 1950 میں برکلے یونیورسٹی کیلیفورنیا سے سیاسیات میں گریجویشن کی۔ 1952ء میں آکسفورڈ یونیورسٹی سے اصول قانون میں ماسٹر کی ڈگری لی۔ اسی سال مڈل ٹمپل لندن سے بیرسٹری کا امتحان پاس کیا۔ پہلے ایشیائی تھے جنھیں انگلستان کی ایک یونیورسٹی ’’ساؤ تھمپئین‘‘ میں بین الاقوامی قانون کا استاد مقرر کیا گیا۔ کچھ عرصہ مسلم لاء کالج کراچی میں دستوری قانون کے لیکچرر رہے۔ 1953ء میں سندھ ہائی کورٹ میں وکالت شروع کی۔ دسمبر 1967ء میں پاکستان پیپلز پارٹی کی بنیاد رکھی۔ 1970ء کے عام انتخابات میں پیپلز پارٹی نے مغربی پاکستان میں نمایاں کامیابی حاصل کی۔ دسمبر 1971ء میں جنرل یحیٰی خان نے پاکستان کی عنان حکومت مسٹر بھٹو کو سونپ دی۔ وہ دسمبر 1971ء تا 13 اگست 1973 صدر مملکت کے عہدے پر فائز رہے۔ 14 اگست 1973ء کو نئے آئین کے تحت وزیراعظم کا حلف اٹھایا۔ 1977ء کے عام انتخابات میں دھاندلیوں کے سبب ملک میں خانہ جنگی کی سی کیفیت پیدا ہو گئی۔ 5 جولائی 1977 کو جنرل محمد ضیا الحق نے مارشل لاء نافذ کر دیا۔ ستمبر 1977ء میں مسٹر بھٹو نواب محمد احمد خاں کے قتل کے الزام میں گرفتار کر لیے گئے۔ 18 مارچ 1978ء کو ہائی کورٹ نے انھیں سزائے موت کا حکم سنایا۔ 6 فروری 1979ء کو سپریم کورٹ نے ہائی کورٹ کے فیصلے کی توثیق کر دی۔ 4 اپریل کو انھیں راولپنڈی جیل میں پھانسی پر لٹکا دیا گیا۔",
            text_b="حکومت عوام کی اور عوام کے لئے قائم کرنا اور اِن کے منہ میں زبان دینا ہے ۔یہ خیالات تھے پاکستان کے چوتھے صدر ذوالفقار علی بھٹو کے جو کہ پاکستان پیپلز پارٹی کے چیئر مین اور سابق وزیرِ خارجہ تھے ۔ ذوالفقار علی بھٹو دنیا کی قدیم ترین تہذیبوں میں سے ایک موہنجوداڑو کی سرزمین لاڑکانہ میں 5جنوری 1928ء کو پیدا ہوئے ۔ اِن کے والد سر شاہ نواز بھتو بھی دیاست کے میدان سے وابستہ رہے تھے ۔ آپ نے ہائی سکول تک تعلیم بمبئی سے حاصل کی ۔ سیاسیات میں گریجوایشن کی ڈگری برکلے یونیورسٹی کیلیفورنیا سے 1950ء میں کی ۔ اور 1952ء میں ماسٹر اصول قانون میں آکسفورڈ یونیورسٹی سے حاصل کیا ۔ پھر بیرسٹری کا امتحان بھی اسی سال مڈل ٹیمپل لندن سے کیا ۔ آپ کو یہ اعزاز بھی حاصل ہے کہ آپ پہلے ایشیائی تھے جن کو انگلستان کی ساؤتھپیئن یونیورسٹی میں بین الاقوامی قانون کا استاد مقرر ہوئے پھر آُپ پاکستان واپس آگئے ۔ پہلے آ پ وکالت کرتے رہے پھر آ پ سیاست میں آ گئے ۔ آپ نے 1967ء میں پیپلز پارٹی کی بنیاد رکھی ۔ 1970ء کے عام انتخابات میں پیپلز پارٹی نے انتخابات میں واضح کامیابی حاصل کی اور 1971ء میں ملک کی بھاگ دوڑ آ پ کے حوالے کر دی گئی۔دسمبر 1973ء تک آ پ صدر رہے 14 اگست 1973ء کو آپ نے نئے آئین کے مطابق وزیرِاعظم کا حلف اٹھایا ۔آپ نے پہلی مرتبہ نچلء طبقات کے لئے سیاست کے دروازے کھول دیئے انہوں نے عوام میں بنیادی حقوق کا شعور اور حقیقی مساوات و آزادی کی محبت پیدا کی اپنی جدوجہد کے نتیجے میں وہ پاکستانی سیاست کی ماہیت و کیفیت میں انقلابی تبدیلیاں رونما کرنے میں کامیاب رہے۔ 5جولائی 1977ء میں جنرل ضیاء الحق نے مارشل لاء نافز کر کے ذوالفقار علی بھٹو کو نواب احمد خان کے قتل کے الزام میں ستمبر 1977ء کو گرفتار کر لیا گیا ۔ راولپنڈی کی جیل میں 4اپریل 1979ء کو پھانسی دے دی گئی ۔ ااور آ پ کا قول امر ہو گیا ۔ عوامی سیاست عوام کے ذریعے عوام کے لئے",
            label=0
        ),
        InputExample(
            guid=26,
            text_a="پاکستانی سائسندان پاکستانی ایٹم بم کے خالق ہندوستان کے شہر بھوپال میں پیدا ہوئے ۔ ڈاکٹر قدیرخان پندرہ برس یورپ میں رہنے کے دوران ہالینڈ سے ماسٹرز آف سائنس جبکہ بیلجیئم سے ڈاکٹریٹ آف انجینئرنگ کی اسناد حاصل کرنے کے بعد 31 مئی 1976ء میں انہوں نے ذوالفقار علی بھٹو سے مل کر انجینئرنگ ریسرچ لیبارٹریزمیں شمولیت اختیار کی ۔ اس ادارے کا نام یکم مئی 1981ء کو جنرل ضیاء الحق نے تبدیل کر کے ’ ڈاکٹر اے کیو خان ریسرچ لیبارٹریز ‘ رکھ دیا ۔ڈاکٹر قدیر خان پر ہالینڈ کی حکومت نے اہم معلومات چرانے کے الزامات کے تحت مقدمہ بھی دائر کیا جن معلومات کو چرانے کی بنا پر مقدمہ داخل کیا گیا ہے وہ عام اور کتابوں میں موجود ہیںـ جس کے بعد ہالینڈ کی عدالت عالیہ نے ان کو باعزت بری کر دیا تھا ۔ مئی 1998ء میں پاکستان نے بھارتی ایٹم بم کے تجربے کے بعد کامیاب تجربہ کیا ۔ ـڈاکٹر قدیر خان کو وقت بوقت 13 طلائی تمغے ملے ۔ ڈاکٹر قدیرخان نے ہالینڈ میں قیام کے دوران ایک مقامی لڑکی ہنی خان سے شادی کی اور جن سے ان کی دو بیٹیاں ہوئیں ۔2004 ان پر یٹمی آلات دوسرے اسلامی ممالک کو فروخت کرنے کا الزام لگا ۔ جس کے جواب میں انہوں نے یہ ناقابل یقین اعتراف کیا کہ انہوں نے ایسا کیا ہے اور حکومت پاکستان کا یا فوج کا اس میں کوئی ہاتھ نہیں رہا ۔ اس کے بعد قدیر خان کو نظر بند کر دیا گیا ۔ اگست 2006ء میں انکشاف ہوا کہ قدیر خان کو کینسر ہے ۔",
            text_b="عبد القدیر خان یکم اپریل 1936 کو بھوپال کے ایک متمول مدہبی خاندان میں چشم افروز ہوئے ۔والد ماجد خان کا تعلق درس و تدریس سے تھا۔معلم اور سائنسدان ڈاکٹر عبدالقدیر خان کی ڈگری بی ایس سی بشیر سید کے زیر ِ سایہ علوم طبیعات اور شماریات میں حاصل کی ،دل و دماغ مطمئن نہ ہوا ۔ چنانچہ 1956 میں جامعیہ میں دوسری میٹلرجی کراچی کا رُخ کیا اور علوم ِ دھات سازی کی ڈگری اعلی نمبروں سے حاصل کی۔simes ان کی خداداد صلاحیت اور قابلیت کو مانتے ہوئے عالمی انجینرنگ ادارے نے انہیں جرمنی جا کر اعلی تعلیم کے لئے اسکالر شپ ۔چنا نچہ 25سالہ نوجوان قدیر خان نے ٹیکنیکل یونیورسٹی برلن جا کر میٹلرجی کی تعلیم شروع کی یہاں اِ ن کی ملاقات ایک ایم ایس سی انجینرنگ میں ہونہار ،محقق ،ڈچ خاتون 'مس ہینی ' سے ہوئی اور دونوں رشتہ ازدواجی میں بندھ گئے ۔بیگم کے مشورے سے دونوں نے ہالینڈ آکر ممتاز تحقیقی ادارے ڈیلفٹ یونیورسٹی میں داخلہ لے لیا یہاں عبد القدیر خان نے طبیعات ،ریاضی اور میٹالرجی میں چار سال تحقیق کے بعد ایم ایس سی کی ڈگری حاصل کی ۔ذوالفقار بھٹو نے پاکستان بھر کے زیرک سائنسدانوں کو جمع کر کے کینیڈا کی معاونت سے پاکستان میں ایٹمی کراچی میں ایٹمی توانائی کی دسترس کے لئے پی ۔ای ۔سی توانائی کمیشن پر پُر زور کوششیں شروع کر دیں ۔جس میں خاطر خواہ ترقی نہ ہو سکی ۔پاکستان میں نیوکلیائی طاقت حاصل کرنے کے ارتقاء کا عبد القدیر خا ن بغور مطالعہ کر رہے تھے۔جولائی 1976میں بھٹو صاحب سے بالواستہ ملاقات کر کے ڈاکٹر نے اپنی خدمات پاکستان کے لئے پیش کر دیں اور اس طرح ایک نئی تاریخ رقم کر دی۔شمالی پاکستان میں پہاڑوں کے درمیان مخلوط شہر کہوٹہ میں خان ریسرچ لیبارٹری کو تب کے۔آر ۔ایل لیبارٹریز کے نام سے شروع کیا گیا۔بعد میں چند باتیں سامنے آگئیں۔ جس میں سے ایک بات کا خود اقرار ڈاکٹر عبدالقدیر خان نے اپنے منہ سے کیا کہ وہ ایٹمی آلات اسلامی ممالک میں فروخت کرتے ہیں اور فوج یا حکومت کا اِس میں کوئی عمل دخل نہیں ۔ اِ س کے بعد اُن کو نظر بند کر دیا گیا۔اِس کے بعد ہی اُن کی کینسر کی بیماری بھی سامنے آئی۔",
            label=0
        ),
        InputExample(
            guid=27,
            text_a="تقریباً 25 سال کی عمر میں آپ صلی اللہ علیہ و آلہ وسلم نے شام کا دوسرا بڑا سفر کیا جو حضرت خدیجہ علیہا السلام کے تجارتی قافلہ کے لیے تھا۔ حضرت محمد صلی اللہ علیہ و آلہ وسلم نے ایمانداری کی بنا پر اپنے آپ کو ایک اچھا تاجر ثابت کیا۔ آپ صلی اللہ علیہ و آلہ وسلم دوسرے لوگوں کا مال تجارت بھی تجارت کی غرض سے لے کر جایا کرتے تھے۔ آپ صلی اللہ علیہ و آلہ وسلم یہ خدمات حضرت خدیجہ علیہا السلام کے لیۓ بھی انجام دیا کرتے تھے۔ اس سفر سے واپسی پر حضرت خدیجہ کے غلام میسرہ نے ان کو حضور صلی اللہ علیہ و آلہ وسلم کی ایمانداری اور اخلاق کی کچھ باتیں بتائیں۔ انہوں نے جب یہ باتیں اپنے چچا زاد بھائی ورقہ بن نوفل کو بتائیں تو ورقہ بن نوفل نے کہا کہ جو باتیں آپ نے بتائیں ہیں اگر صحیح ہیں تو یہ شخص یقیناً نبی ہیں. آپ حضرت محمد صلی اللہ علیہ و آلہ وسلم کے اچھے اخلاق اور ایمانداری سے بہت متاثر ہوئیں اور آپ صلی اللہ علیہ و آلہ وسلم کو شادی کا پیغام دیا جس کو آپ صلی اللہ علیہ و آلہ وسلم نے حضرت ابوطالب کے مشورے سے قبول کر لیا۔ اس وقت آپ صلی اللہ علیہ و آلہ وسلم کی عمر 25 سال تھی۔حضرت خدیجہ علیہا السلام قریش کی مالدار ترین اور معزز ترین خواتین میں سے تھیں۔ حضرت خدیجہ علیہا السلام سے آپ صلی اللہ علیہ و آلہ وسلم کی چار بیٹیاں حضرت زینب رضی اللہ تعالٰی عنہ ، حضرت رقیہ رضی اللہ تعالٰی عنہ ، حضرت ام کلثوم رضی اللہ تعالٰی عنہ اور حضرت فاطمہ رضی اللہ تعالٰی عنہ پیدا ہوئیں۔ بعض علمائے اہلِ تشیع کے مطابق حضرت فاطمہ علیہا السلام کے علاوہ باقی بیٹیاں حضرت خدیجہ علیہا السلام کے پہلے خاوند سے تھیں۔",
            text_b="لو گو ں کی بھلائی اور ہدایت کے لیے دنیا میں بے شمار نبی اور پیغمبر آئے، جنھوں نے اللہ کے بتائے ہوئے طریقے کے مطابق لوگو ں کو زندگی بسر کر نے کی نصیحت کی۔ سب سے آخر میں ہمارے پیارے نبی حضر ت محمد دنیا میں تشر یف لا ئے۔ آپ کے بعد کسی پیغمبر یا نبی کے دنیا میں آنے کی ضرورت نہیں رہی، کیو نکہ آپ پر دین اسلا م مکمل ہو گیا۔ آپ ﷺ پر قرآن مجید نازل ہوا جو ایک مکمل زریعہ ہدا یت ہے۔ہمارے پیارے نبی عرب کے معزز قبیلے قر یش سے تعلق رکھتے تھے۔ آپ کے والد کا نام حضرت عبداللہ تھا۔ ہمارے پیارے نبی 22 اپر یل 571ء کو مکے میں پیدا ہو ئے ۔ آپ کے والد آپ کی ولادت سے چند ماہ قبل فوت ہو گئے تھے۔آپ کے دادا حضرت عبدالمطلب نے آپ کا نام محمدﷺ رکھا۔ عرب کے رواج کے مطا بق بچوں کی پرورش کھلے ماحول میں ہوتی تھی۔چنانچہ آپ کی پرورش مائی حلیمہؓ نے کی۔ ہما رے پیا رے نبی شروع ہی سے اچھی عادات اور اعلیٰ اخلاق کے مالک تھے۔ بچپن ہی سے آپ کو کھیل تماشے سے کوئی دلچسپی نہ تھی۔ جوانی میں بھی آپ نے بڑی پا کیزہ زندگی بسر کی۔آپ تمام برائیوں سے دور رہتے اور نیکی کی تعلیم دیتے۔آپ نہایت سچے اور ایماندار تھے اسی لیے لوگ آپ کے پاس امانتیں رکھتے اور آپ کوصادق اور امین کہہ کر پکا رتے تھے۔آپ کی ان خوبیو ں کی وجہ سے مکے کی مالدار خاتون حضرت خد یجہؓ نے جو طاہرہ کے لقب سے مشہو ر تھیں۔آپ اپنا مال ِ تجارت دے کر ملک شا م بھیجا۔وہ آپ کی امانداری اور اخلا ق کی وجہ سے بہت متا ثر ہوئیں ۔کہ آپ سے شادی کی خواہش کی۔ جو آپ نے قبول فرما لی۔آپ عبادت کے لیے غارِحرا تشریف لے جاتے تھے۔چالس سال کی عمر میں آپ کو نبوت عطا ہوئی۔آپ نہایت سادہ طبیعت کے مالک تھے۔آپ بہت رحمدل تھے۔آپ ثابت قدمی سے اسلام کی تبلیغ کرتے رہے ۔وہ وقت بھی آگیا جب لوگ اسلام قبول کرنے لگے۔آپ نے اللہ کی راہ میں کئی جنگیں لڑیں ۔آخرکار آپ 12 ربیع الاول 11ھ کو تریسٹھ سال کی عمر میں آپ کا وصال ہو گیا۔آپ کا روضہ مبارک مدینہ منورہ میں ہے۔",
            label=0
        ),
        InputExample(
            guid=28,
            text_a="بلھے شاہ کا اصل نام عبد اللہ شاہ تھا۔ 1680ء میں مغلیہ سلطنت کے عروج میں اوچ گیلانیاں میں پیدا ہوئے کچھ عرصہ یہاں رہنے کے بعد قصور کے قریب پانڈو میں منتقل ہو گئے۔ ابتدائی تعلیم یہیں حاصل کی۔ ان کا تعلق صوفیاء کے قادریہ مکتبہ فکرسے تھا۔ ان کی ذہنی نشوونما میں قادریہ کے علاوہ شطاریہ فکر نے بھی نمایاں کردار ادا کیا تھا۔ اسی لئے ان کی شاعری کے باغیانہ فکر کی بعض بنیادی خصوصیات شطاریوں سے مستعار ہیں۔ صوفیانہ مسائل پر گہری نظر رکھتے تھے اور قادریہ سلسلے سے بھی بیعت تھے اس لئے ان کی ذات میں یہ دونوں سلسلے مل کر ایک نئی ترکیب کا موجب بنے۔ بلھے شاہ انہی شاہ عنایت کے مرید تھے۔ بلھے شاہ کی شاعری عالمگیری عقیدہ پرستی کے خلاف رد عمل ہے۔ ان کی شاعری میں صلح کل، انسان دوستی، اور عالم گیر محبت کا جود رس ملتا ہے بلھے شاہ کا انتقال 1757ء میں قصور میں ہوا اور یہیں دفن ہوئے۔ ان کے مزار پر آج تک عقیدت مند ہر سال ان کی صوفیانہ شاعری گا کر انہیں خراج عقیدت پیش کرتے ہیں۔",
            text_b="اسلام کی اشاعت میں بہت سے اولیائے اکرام اور اعظیم شخصیات آئیں جنھوں نے اسلام کو وسیع و عریض بنانے کے لیے اہم کام سرانجام دیے۔اِن اولیائے اکرام کے ذریعے اِس کو روشن اور منور کیا۔بلھے شاہ اْچ میں 1680ء کو پیدا ہوئے ۔اْچ چھور کر ساہیوال میں رہے اِنھوں نے اِبتدائی تعلیم قصور سے حاصل کی ۔بلھے شاہ کا بچپن بھی اِسی گاؤں میں مویشوں کو چرانے میں گزر گیا۔اِن پرالزام بھی لگا کہ اِنھوں نے مویشوں کے ذریعے جیون خان کی فصل تباہ کر دی لیکن بلھے شاہ نے اِن کی فصل ہری بھری کر دی۔ آپ بہت زیادہ ذہین تھے انھوں نے قرآن واحدیث کا بہت مطالعہ کیا۔وہ شاہ عنایت کے مرید تھے ۔وہ قصور میں حافظ غلام مرتضٰی آئے وہاں سے ہی مروجہ علوم حاصل کیا۔آپ نے باطنی علوم کے ساتھ ساتھ روحانی علوم بھی حاصل کیا۔وہ شاہ عنایت کے مرید سید عبدالحکیم کے کہنے پر ہوئے اور انھوں نے کہا کہ وہ جو بھی تعلیمات دیں اْنھیں غور سے سن کر عمل کرنا ۔بلھے شاہ نے یہاں پر بہت سے امتحانات پاس کرنے بعد بہت کرنے کی اجاذت لی اور اِس عشق نے اِن کو عشقِ حقیقی تک پہنچا دیا۔بلھے شاہ اونگزیب کی تنگ نظری کو بھی دیکھا قصور کو بار بار تباہ ہوتے بھی دیکھا۔ وہ زندگی میں کشمکش اور انسانوں کے ساتھ سلوک اور نفرتوں کے پہاڈ کو دیکھتے تھے۔اپنی شاعری کے ذریعے لوگوں کے دِلوں کو مضبوط کرنے کے لیے کوشاں رہے اور جس محفل میں جاتے دوسروں کی قوالیاں سنتے اپنی قوالی کے بعد دعا کرتے ۔اللہ اور اس کے حضورکے مطابق امن وسکون کے لیے لوگوں کو اپنے شاعری سے مسترد کرتے تھے۔بلھے شاہ کی شاعری خاص اثر رکھتی ہے اور آج بھی دنیا میں اِن کو بہت اہمیت حاصل ہے ۔اِن کی شاعری سے بہت سے لوگ متاثر ہوتے۔اِن کی وفات 1757ء میں قصور میں ہی ہوئی۔بلھے شاہ کی وفات کے بعد بھی آج بھی وہ شاعری کی دنیا میں ذندہ ہیں اور اِن کو بہت سراہا جاتاہے۔",
            label=0
        ),
        InputExample(
            guid=29,
            text_a="بینظیر بھٹو پاکستان کی پہلی خاتون وزیراعظم تھیں۔ پہلی بارآپ 1988ء میں پاکستان کی وزیراعظم بنیں لیکن صرف 20 مہینوں کے بعد اس وقت کے صدر پاکستان غلام اسحاق خان نے بے پناہ بدعنوانی کے باعث اپنے خصوصی اختیارت کو استعمال کرتے ہوئے اسمبلی کو ختم کرتے ہوئے نئے الیکشن کروائے۔ بینظیر بھٹو کا تعلق پاکستان پیپلز پارٹی سے تھا۔ ذوالفقار علی بھٹو کی بڑی صاحبزادی سابق وزیراعظم بینظیر بھٹو 21 جون 1953 میں سندھ کے مشہور سیاسی بھٹو خاندان میں پیدا ہوئیں۔ ابتدائی تعلیم کراچی میں حاصل کی Lady Jennings Nursery School اور Convent of Jesus and Mary انھوں نے 15 سال کی عمر بے نظیر بھٹو نے 1973 پولیٹیکل سائنس میں گریجوایشن کر لیا۔ اس کے بعد انھوں نے آکسفورڈ یونیورسٹی میں داخلہ لیا جہاں سے انہوں نے فلسفہ، معاشیات اورسیاسیات میں ایم اے کیا۔ بے نظیر لیاقت باغ میں عوامی جلسے سے خطاب کرنے کے بعد اپنی گاڑی میں بیٹھ کر اسلام آباد آرہی تھیں اس دوران جب وہ پارٹی کارکنوں کے ساتھ اظہار یکجہتی کے لیے گاڑی کی چھت سے باہر نکل رہی تھیں کہ نامعلوم شخص نے ان پر فائرنگ کر دی۔ اس کے بعد بینظیر کی گاڑی سے کچھ فاصلے پر ایک زوردار دھماکہ ہوا اس دھماکے میں بینظیر بھٹو جس گاڑی میں سوار تھیں، اس کو بھی شدید نقصان پہنچا لیکن گاڑی کا ڈرائیور اسی حالت میں گاڑی کو بھگا کر راولپنڈی جنرل ہسپتال لے گیا جہاں وہ زخموں کی تاب نہ لاتے ہوئے جاں بحق ہو گئیں۔",
            text_b="بینظیربھٹو 1953ء کو کراچی میں پیدا ہو ئیں۔73-1969ء میں کیمرج میں ریڈ کلف کالج میسا چو سٹس سے بی اے کی ڈگری لی ۔77-1973 ء آکسفورڈیونیورسٹی میں زیر تعلیم رہیں۔ 77-1971ء بینظیر کے والد زوا لفقار علی بھٹو وزیراعظم رہے۔1977ء جنرل ضیالحق زوالفقار کی منتخب حکو مت بر طرف کر دی۔1979 ء زوالفقار علی بھٹوکو پھانسی دے دی گئی۔1984ء میں بینظیر کو ملک سے با ہر جانے کی اجازات می۔اور وہ بر طانیہ چلے گئیں۔اور اپنے والد کی پیپلزپارٹی جانشین رہنما بن گئیں۔اگست1988 ء میں پیپلز پارٹی نے تمام انتخابات میں اکژیت حاصل کرلی۔1988ء کو دسمبر محترمہ بینظیربھٹونے تاریح کی سب سے کم عمراور عالم اسلام کی پہلی حاتون وزیراعظم کی حشیت سے حلف اٹھایا۔1993ء میں پیپلز پارٹی دوبارہ انتخابات میں کامیابی ہو ئی۔اپنے والد کے بعد وہ دوسری شخصت تھیں۔جو دوسری مرتبہ وزارت عظمٰی کے عہدے پر فائز ہوتٗیں تھیں ۔1997ء کے عام انتخابات میں پاکستان پیپلز پارٹی کو ملک کی تاریخ کی بد تر ین شکست کا سا منا کرنا پڑا اِس وقت آپ د بئی میں مقیم تھیں۔ وہ 2008ء میں منعقد ہونے والے عا م انتخا با ت میں ا پنی پارٹی کی قیادت کا فریضہ انجا م دے رہی تھیں اور ملک کے مختلف شہروں میں جلسہ عا م سے خطا ب کر رہی تھیں۔",
            label=0
        ),
        InputExample(
            guid=30,
            text_a="ڈاکٹر عافیہ صدیقی پاکستان سے تعلق رکھنے والی سائنسدان ہے جسے امریکی حکومت نے 2003ء میں اغوا کر کے غیرقانونی طور پر قید کیا ہؤا ہے۔ ڈاکٹرعافیہ صدیقی پیدائش مارچ 2 1972 کراچی میں پیدا ہوئی۔ 8 سال کی عمر تک زیمبیا میں تعلیم حاصل کرنے کے بعد کراچی میں ابتدائی اور ثانوی تعلیم مکمل کی 2002 میں پاکستان واپس آئیں مگرملازمت نہ ملنے کی وجہ سے امریکہ ملازمت ڈھونڈنے کے سلسلہ میں دورہ پر گئیں۔ اس دوران میریلینڈ میں ڈاک وصول کرنے کے لیے ڈاک ڈبہ کرائے پرلیا اور 2003 میں کراچی واپس آ گئیں۔ FBI نے شک ظاہر کیا کہ یہ ڈاک ڈبہ دراصل القاعدہ سے تعلق رکھنے والے کسی شخص کے لیے کرائے پر لیا گیا تھا۔ امریکی ابلاغ میں عافیہ صدیقی کی بطور دہشت گرد تشہیر کی گئی۔ 30 مارچ 2003 کو اپنے تین بچوں سمیت راولپنڈی جانے کے لیے ٹیکسی میں ہوائی اڈہ کی طرف روانہ ہوئی مگر راستے میں پاکستانی خفیہ ادارے نے بچوں سمیت عافیہ کو اغوا کرکے امریکی فوجیوں کے حوالے کر دیاعالمی اداروں نے خیال ظاہر کیا کہ افغانستان میں امریکی جیل بگرام میں قیدی نمبر 650 شاید عافیہ صدیقی ہی ہے جو وہاں بےحد بری حالت میں قید تھی۔ پاکستانی اخبارات میں شور مچنے کے بعد امریکیوں نے اچانک اعلان کیا کہ عافیہ کو 27 جولائی 2008 کو افغانستان سے گرفتار کر کے نیویارک پہنچا دیا گیا ہے تاکہ ان پردہشت گردی کے حوالہ سے مقدمہ چلایا جا سکے۔ انسانی حقوق کی تنظیموں نے امریکی کہانی کو ناقابلِ یقین قرار دیا۔ افغانستان میں امریکی فوجیوں نے دوران گرفتاری عافیہ کو گولیوں کا نشانہ بنا کر شدید زخمی کر دیا. تاہم امریکیوں نے الزام لگایا کہ عافیہ نے امریکی فوجی کی بندوق اٹھانے کی کوشش کی تھی جس پر انھوں نے اس پر گولیاں چلا دیں۔",
            text_b="2مارچ 1972 میں پیدا ہونے والی عافیہ صدیقی کراچی کے ایک بہت معزز گھرانے سے تعلق رکھتی ہیں۔ اِن کے والد کا نام محمد اور والدہ کا نام عصمت ہے۔ والد خود ڈاکٹر تھے تو انہی کی خواہش تھی کہ اُن کی بیٹی بھی ڈاکٹری کی تعلیم حاصل کرے اِس لیے وہ اِن کی تعلیم پر بہت توجہ دیتےتھے۔ ابتدائی تعلیم کے بعد عافیہ نے 1990 میں امریکہ میں سکونت اختیار کرلی۔ عافیہ شروع سے ہی مذہبی رجحان رکھتی تھی امریکہ میں بھی انہوں نے بہت امدادی تنظیموں کو جوائن کیا تھا۔ امریکہ یونیورسٹی برانڈیز میں داخل ہوئی اور نیروسائنس میں پی ۔ایچ ۔ڈی مکمل کی۔ 1995 میں ان کا نکاح کراچی کے باسی ڈاکٹر امجد خان سے ہو گیا۔ ڈاکٹریٹ کرنے کے بعد عافیہ کو امریکہ ہی میں جاب مل گئی ان کے شوہر جن سے اِن کا فون پر نکاح ہوا تھا وہ بھی امریکہ آگئے ان میں سے ان کے تین بچے تھے۔ امریکہ میں قیام کے دوران آپ نے ڈاک کے لیے ایک ڈاک ڈبہ بھی بک کروا رکھا تھا۔ 2003 میں عافیہ اپنے تین بچوں کے ساتھ کراچی اپنے آبائی گھر پہنچیں چند دنوں کے بعد انہوں نے اپنی والدہ اور ہمشیرہ کو بتایا کہ وہ ضروری کام سے اسلام آباد جا رہی ہیں مقررہ دن انہوں نے ٹیکسی منگوائی تینوں بچوں کے ساتھ ٹیکسی میں روانہ ہوئی کہ وہ راستے میں سے ہی غائب ہو گیئں۔ منظر عام سے غائب ہونے کے بعد مختلف قسم کی قیاس آرائیاں لگائی گئیں۔ عافیہ صدیقی 2003 سے 2008 تک افغانستان میں واقع بگرام جیل میں قیدی نمبر 650 کے نام سے زیر ِحراست رہی اور وہاں انہیں ہر طرح کے ظلم و ستم کا نشانہ بنایا گیا۔ عافیہ صدیقی پر لگائے گئے بیشتر الزامات مفروضوں پر مبنی ہیں جبکہ آخر ی الزام امریکی فوجیوں پر فائرنگ کا ہے کہ جس میں کوئی امریکی فوجی زخمی بھی نہیں ہوا اور نہ ہی کوئی ایسی شہادت عدالت میں پیش کی گئی۔ اِس سے یہ بات ثابت ہوتی ہے کہ عافیہ کو سزا ہونا انصاف نہیں بلکہ جیوری نے یہ فیصلہ صرف اور صرف تعصب کی بنیاد پر دیا ہے۔",
            label=0
        ),
        InputExample(
            guid=31,
            text_a="حضرت زینب سلام اللہ علیہا امام علی علیہ السلام اور حضرت فاطمہ سلام اللہ علیہا کی بیٹی یعنی حضرت محمد صلی اللہ علیہ و آلہ و سلم کی نواسی تھیں۔ وہ 5 جمادی الاول 6ھ کو مدینہ میں پیدا ہوئیں۔ واقعہ کربلا کے بعد حضرت زینب سلام اللہ علیہا کا کردار بہت اہم ہے۔ واقعہ کربلا کے بعد وہ دمشق لے جائی گئیں جہاں یزید کے دربار میں دیا گیا ان کا خطبہ بہت مشہور ہے۔ آپ نے اپنے خطبے میں فرمایا۔ یزيد اگر چہ حادثات زمانہ نے ہمیں اس موڑ پر لا کھڑا کیا ہے اور مجھے قیدی بنایا گیا ہے لیکن جان لے میرے نزدیک تیری طاقت کچھ بھی نہیں ہے۔ خدا کی قسم، خدا کے سوا کسی سے نہیں ڈرتی اس کے سوا کسی اور سے گلہ و شکوہ بھی نہیں کروں گی۔ ہم اہل بیت پیغمبر سے دشمنی کے لئے تو جتنی بھی سازشیں کرسکتا ہے کر لے لیکن خدا کی قسم تو ہمارے نام کو لوگوں کے دل و ذہن اور تاریخ سے نہیں مٹا سکتا اور اسی طرح تو اپنے دامن پر لگے ننگ و عارکے بدنما داغ کو بھی نہیں دھو سکتا، خدا کی نفرین و لعنت ہوظالموں اور ستمگروں پر۔ حضرت زینب اپنے بھائی امام حسین کی شہادت کے تقریبا' ڈیڑھ سال بعد 15 رجب المرجب 62 ہجری کودرجہ شہادت پر فائز ہوئیں۔ ان کا روضہ اقدس شام کے دارالحکومت دمشق میں ہے۔",
            text_b="حضرت بی بی زینب سلام اللہ امام علی ؑ اور حضرت فاطمہؓ کی بڑی بیٹی اور حضرت محمد کی نواسی ہیں۔ آپ کا نام زینبؓ اور کنیت ام اِلحسن ہے۔ روایت ہے کہ رسول نے اس نومولود کو طلب کیا۔ اْسے سنتے سے لگایا اور نام زینبؓ رکھا۔ حضرت زینبؓ کو حضرت محمد کی زیارت اور سیکھنے کا موقع ملا۔ جب وہ سات سال کی تھیں تو ان کی نانا حضرت محمد کا انتقال ہو گیا۔ اس کے تقریباَ تین ماہ بعد ان کی والدہ کا انتقال ہو گیا تھا۔ حضرت زینبؓ کی شادی حضرت عبداللہ بن جعفر طیارؒ سے ہوئی۔ ان کے پانچ بچے ہوئے۔ جن میں سے حضرت عون اور حضرت محمد کربلا میں حضرت امام حسنؓ کے ساتھ شہید ہو گئے۔ حضرت زینبؓ حضرت فاطمہؓ حضرت علیؓ کی ترتیب گاہ کا موتی ہیں۔ حضرت زینبؓ کا کردار امام حسینؓ کے کردار کا مظہر ہے۔ اور یہ ہر دور کی عورت کے لیے بہترین نمونہ ہے۔ خود حضرت زینبؓ کربلا میں موجود تھیں۔ واقعہ کربلا کے بعد آپؓ کے کردار اور اہمیت کے حامل خاص اہمیت رکھتا ہے۔ حضرت زینبؓ نے لوگوں کو سچائی سے آگاہ کیا۔ آپؓ کے خطبہ کے سبب ایک انقلاب آیا۔ حضرت زینبؓ نے جس پاکیزہ ابتدا میں بنی امیہ کا کاخِ ضرمر اپنے بیان کی فوت سے ریزہ ریزہ کردیا۔ اس کے سامنے یزید کا سرندامت وشرمندگی سے جھک گیا۔ اور اسے رسول داری کا استدلال اس طرح رسوا کر گیا کہ وہ سوچنے لگا کہ اے کاش زمین کا منہ کھل جائے اور وہ اس میں چھپ کر اپنی رہی سہی عزت بچا سکے۔ مگر اب وقت گزر چکا تھا۔ اور اموی خاندان کی طاغونیت اور فرعونیت کا بت پاش پاش ہو چکا تھا۔ اور ان کی سطوت وامریث کے چراغ گل ہو چکے تھے۔ اور اب یزید کے ایوان اقتدار پر مایوسی ،پریشانی اضطراب اور ندامت کے سائے تھے۔ حضرت زینبؓ نے لوگو ں کو سچائی سے آگاہ کیا۔ آپؓ اپنے بھائی امامِ حسینؓ کی شہادت کے تقریباَ ڈیڑھ سال بعد 15رجب المرجب 62 ہجری کو دوسری مرتبہ سفرِ شام کے دوران درجہ شہادت پر فائز ہوئیں۔ آپ کا روضہ مبارک شام کے دارالحکومت دمشق میں آج بھی مرجہ خلائق ہے۔ دعا ہے کہ اللہ تعالیٰ حضرت زینبؓ کی طرح ہماری خواتین کو ان کے راستے پرچلنے کی توفیق دے آمین",
            label=0
        ),
    
]


In [5]:
### Classes ###
classes = ['P', 'NP']

### Label Map ###
label_map = {'P': 1, 'NP': 0}



# # # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
# _, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # # Step 2: Manually load XLM-RoBERTa model/tokenizer
# model_name = "xlm-roberta-base"
# tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
# plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized fr

In [6]:
template = ManualTemplate(
    text = '{"placeholder":"text_a"} اور {"placeholder":"text_b"} کیا ایک جیسے ہیں؟ {"mask"}',
    tokenizer=tokenizer,
)






templates = [
    ("T1", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کیا ایک جیسے ہیں؟ {"mask"}',
        tokenizer=tokenizer,
    )),

    ("T2", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} میں معنی کی مطابقت {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("T3", ManualTemplate(
        text='کیا یہ دونوں جملے {"placeholder":"text_a"} اور {"placeholder":"text_b"} ایک دوسرے کی تکرار ہیں؟ {"mask"}',
        tokenizer=tokenizer,
    )),

    ("T4", ManualTemplate(
        text='{"placeholder":"text_a"} کے مطابق {"placeholder":"text_b"} {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("T5", ManualTemplate(
        text='دونوں جملے {"placeholder":"text_a"} اور {"placeholder":"text_b"} کا تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("T6", ManualTemplate(
        text='{"placeholder":"text_b"} اور {"placeholder":"text_a"} ایک جیسے خیال رکھتے ہیں؟ {"mask"}',
        tokenizer=tokenizer,
    )),

    ("T7", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے الفاظ کی ہم آہنگی {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),

    ("T8", ManualTemplate(
        text='اگر ہم {"placeholder":"text_a"} اور {"placeholder":"text_b"} کو دیکھیں تو یہ {"mask"} ہیں۔',
        tokenizer=tokenizer,
    )),

    ("T9", ManualTemplate(
        text='{"placeholder":"text_a"} کی روشنی میں {"placeholder":"text_b"} {"mask"} سمجھا جا سکتا ہے۔',
        tokenizer=tokenizer,
    )),

    ("T10", ManualTemplate(
        text='{"placeholder":"text_a"} اور {"placeholder":"text_b"} کے بیچ مفہوم کا تعلق {"mask"} ہے۔',
        tokenizer=tokenizer,
    )),
]










verbalizer = ManualVerbalizer(
    classes=classes,
    label_words={
        "P": ["ہم معنی", "ایک جیسے"],    # Paraphrase: same meaning / similar
        "NP": ["مختلف", "غیر مترادف"] 
    },
    tokenizer=tokenizer,
)

In [7]:
# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)

# ==============================
# Training loop with BalancedBatchSampler + random template switching
# ==============================
T = 20   # epochs
K = 1    # steps per prompt
batch_size = 8





prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)
all_epoch_patterns = {}

for epoch in range(T):
    print(f"\n🟦 Epoch {epoch+1}/{T}")

    # Random initial template
    prompt_name, current_template = random.choice(templates)
    epoch_pattern = []

    # Create PromptDataLoader with BalancedBatchSampler
    sampler = BalancedBatchSampler(train_dataset, batch_size=batch_size)
    train_loader = PromptDataLoader(
        dataset=train_dataset,
        tokenizer=tokenizer,
        template=current_template,
        tokenizer_wrapper_class=WrapperClass,
        max_seq_length=128,
        batch_size=batch_size,
        batch_sampler=sampler,
        shuffle=False  # shuffle is ignored when batch_sampler is used
    )

    step_counter = 0
    epoch_loss = 0.0

    for batch in train_loader:
        # Move batch to device
        # batch = {k: v.to(device) for k, v in batch.items()}
    
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        # loss = criterion(logits, batch["label"])
        
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_pattern.append(prompt_name)

        step_counter += 1

        # Switch template every K steps
        if step_counter % K == 0:
            prompt_name, current_template = random.choice(templates)

            # Rebuild PromptDataLoader with new template but same sampler
            train_loader = PromptDataLoader(
                dataset=train_dataset,
                tokenizer=tokenizer,
                template=current_template,
                tokenizer_wrapper_class=WrapperClass,
                max_seq_length=128,
                batch_size=batch_size,
                batch_sampler=sampler,
                shuffle=False
            )

    all_epoch_patterns[f"epoch_{epoch+1}"] = epoch_pattern
    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")
    print(f"Prompt pattern: {epoch_pattern}")


🟦 Epoch 1/20


tokenizing: 0it [00:00, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (516 > 512). Running this sequence through the model will result in indexing errors
tokenizing: 32it [00:00, 228.78it/s]
tokenizing: 32it [00:00, 154.74it/s]
tokenizing: 32it [00:00, 150.92it/s]
tokenizing: 32it [00:00, 147.60it/s]
tokenizing: 32it [00:00, 162.69it/s]


Epoch 1 Loss: 12.8085
Prompt pattern: ['T2', 'T1', 'T5', 'T4']

🟦 Epoch 2/20


tokenizing: 32it [00:00, 204.88it/s]
tokenizing: 32it [00:00, 155.18it/s]
tokenizing: 32it [00:00, 195.73it/s]
tokenizing: 32it [00:00, 154.90it/s]
tokenizing: 32it [00:00, 167.67it/s]


Epoch 2 Loss: 4.3388
Prompt pattern: ['T3', 'T2', 'T9', 'T2']

🟦 Epoch 3/20


...

Epoch 18 Loss: 3.0784
Prompt pattern: ['T1', 'T6', 'T7', 'T5']

🟦 Epoch 19/20


tokenizing: 32it [00:00, 123.09it/s]
tokenizing: 32it [00:00, 95.48it/s] 
tokenizing: 32it [00:00, 78.42it/s] 
tokenizing: 32it [00:00, 125.08it/s]
tokenizing: 32it [00:00, 123.63it/s]


Epoch 19 Loss: 2.6369
Prompt pattern: ['T4', 'T10', 'T6', 'T4']

🟦 Epoch 20/20


tokenizing: 32it [00:00, 87.86it/s] 
tokenizing: 32it [00:00, 90.31it/s] 
tokenizing: 32it [00:00, 128.77it/s]
tokenizing: 32it [00:00, 121.10it/s]
tokenizing: 32it [00:00, 120.97it/s]

Epoch 20 Loss: 2.8097
Prompt pattern: ['T7', 'T8', 'T3', 'T5']


In [8]:
# ==============================
# Helper Function to Read XML
# ==============================
def read_xml_text(path):
    """
    Reads an XML file and returns cleaned text inside <UPPC_document>...</UPPC_document>.
    Removes all tags and converts inner double quotes to single quotes.
    """
    with open(path, "r", encoding="utf-8") as f:
        xml = f.read()
    
    # Extract text between opening and closing <UPPC_document>
    start = xml.find(">") + 1
    end = xml.rfind("</UPPC_document>")
    text = xml[start:end].strip()
    
    # Remove any remaining tags inside the text
    text = re.sub(r"<.*?>", "", text)
    
    # Replace multiple spaces/newlines with single space
    text = re.sub(r"\s+", " ", text).strip()
    
    # Replace double quotes inside text with single quotes
    text = text.replace('"', "'")
    
    return text

# ==============================
# Load Test CSV
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\PARA\test_pairs.csv")

data_folder = r"C:\Users\stdFurqan\Desktop\paft\PARA\data"

# ==============================
# Make InputExamples
# ==============================
eval_dataset = []
for i, row in df.iterrows():
    # Read XML texts
    text_a_path = os.path.join(data_folder, row['source_file'])
    text_b_path = os.path.join(data_folder, row['target_file'])
    
    text_a = read_xml_text(text_a_path)
    text_b = read_xml_text(text_b_path)
    
    eval_dataset.append(
        InputExample(
            guid=i,
            text_a=text_a,
            text_b=text_b,
            label=label_map[row['label']]
        )
    )

# ==============================
# PromptDataLoader
# ==============================
eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,              # your ManualTemplate
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# ==============================
# Print Classification Report
# ==============================
print("\n📊 Paraphrase Urdu Test Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))


tokenizing: 28it [00:00, 124.12it/s]



📊 Paraphrase Urdu Test Classification Report:
              precision    recall  f1-score   support

           P     0.2857    0.1538    0.2000        13
          NP     0.4762    0.6667    0.5556        15

    accuracy                         0.4286        28
   macro avg     0.3810    0.4103    0.3778        28
weighted avg     0.3878    0.4286    0.3905        28



In [ ]:
roberta

📊 Paraphrase Urdu Test Classification Report:
              precision    recall  f1-score   support

           P     0.2500    0.3077    0.2759        13
          NP     0.2500    0.2000    0.2222        15

    accuracy                         0.2500        28
   macro avg     0.2500    0.2538    0.2490        28
weighted avg     0.2500    0.2500    0.2471        28


In [ ]:
mbert

📊 Paraphrase Urdu Test Classification Report:
              precision    recall  f1-score   support

           P     0.2857    0.1538    0.2000        13
          NP     0.4762    0.6667    0.5556        15

    accuracy                         0.4286        28
   macro avg     0.3810    0.4103    0.3778        28
weighted avg     0.3878    0.4286    0.3905        28